# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math
from numpy import log as ln

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'C'
ods_fold = f'{drive}:/Users/Owner/Downloads/테스트/data'

# ❌Load
- 등록&제원
- 저감장치 부착이력
- 법정동코드
- KOSIS

## 등록&제원

In [5]:
# 전체 컬럼명
'''
'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', 
'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', 
'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', 
'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'
'''

"\n'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', \n'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', \n'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', \n'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'\n"

In [4]:
# about 4m 30.7s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
smig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '본거지법정동코드', '차종_등록정보', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형(소분류)', '연료', '배기량', '최초등록일자', '총중량'])
smig.info()

In [ ]:
smig.columns

Index(['본거지법정동코드', '차종_등록정보', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급',
       '차량말소YN', '차종유형(소분류)', '연료', '총중량', '배기량'],
      dtype='object')

In [ ]:
smig['차종유형(소분류)'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [9]:
smig1 = smig.rename(columns={'본거지법정동코드':'법정동코드', '차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})
smig1.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량'],
      dtype='object')

In [10]:
smig1['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25693532
Y     3840515
Name: count, dtype: int64

### 말소제거

In [11]:
smig2 = smig1[smig1['차량말소YN'] == 'N'].reset_index(drop=True)
smig2.shape

KeyboardInterrupt: 

## 저감장치 부착이력(중복차대제거)

In [ ]:
# about 28.3s
# columns = 저감장치종류, TDMS부착여부, 구조변경일자, 부착업체명, 부착일자, 탈거승인일, 차량번호, 차대번호, 저감장치반납여부, 저감장치구분, 삼원촉매장치반납여부
name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28)'
file_name = f'{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '저감장치구분'])
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    809255 non-null  object
 1   저감장치구분  809255 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


In [ ]:
att['저감장치구분'].unique()

array(['엔진교체', '3종', '1종', '2종', '엔진개조', '1종+SCR', '삼원촉매'], dtype=object)

In [ ]:
att.loc[(att['저감장치구분'] == '1종') | (att['저감장치구분'] == '1종+SCR'), '저감장치부착유무'] = '유'

In [ ]:
att['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    512654
유      296601
Name: count, dtype: int64

## 법정동코드

In [ ]:
# about 5.6s
# columns = '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN' 
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name), usecols=['법정동코드', '시도', '시구군'])
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   46342 non-null  int64 
 1   시도      46342 non-null  object
 2   시구군     46319 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


## KOSIS(2021)

In [ ]:
# about 0.2s
name = '용도별_차종별_시군구별_자동차1일평균주행거리(2021)'
file_name = f'{name}.xlsx'
kos = pd.read_excel(os.path.join(ods_fold, file_name))
kos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


## 4등급 result(for DPF유무)

In [17]:
rs = pd.read_csv(os.path.join(ods_fold, '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03).csv'), usecols=['차대번호', 'DPF유무_수정'])
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


C:\Users\Owner\AppData\Local\Temp\ipykernel_19844\1159834493.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  rs = pd.read_csv(os.path.join(ods_fold, '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03).csv'), usecols=['차대번호', 'DPF유무_수정'])


# ❌ 전처리

## 등급 수정

In [ ]:
smig2['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10960244
3      8623130
1      2771395
4      1511214
5      1154733
X       288966
T2      247483
T3      136301
T4          37
T5          29
Name: count, dtype: int64

In [ ]:
smig2['배출가스등급'].unique()

array(['5', 'X', 'T5', '4', '3', 'T4', 'T3', '2', '1', 'T2'], dtype=object)

In [ ]:
smig2.loc[smig2['배출가스등급'] == '1', '등급'] = '1'
smig2.loc[(smig2['배출가스등급'] == '2') | (smig2['배출가스등급'] == 'T2'), '등급'] = '2'
smig2.loc[(smig2['배출가스등급'] == '3') | (smig2['배출가스등급'] == 'T3'), '등급'] = '3'
smig2.loc[(smig2['배출가스등급'] == '4') | (smig2['배출가스등급'] == 'T4'), '등급'] = '4'
smig2.loc[(smig2['배출가스등급'] == '5') | (smig2['배출가스등급'] == 'T5'), '등급'] = '5'
smig2.loc[smig2['배출가스등급'] == 'X', '등급'] = 'X'

In [ ]:
smig2['등급'].value_counts(dropna=False)

등급
2    11207727
3     8759431
1     2771395
4     1511251
5     1154762
X      288966
Name: count, dtype: int64

## 연료 수정

In [ ]:
smig2['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [ ]:
smig2['연료'].unique()

array(['휘발유', '경유', nan, '기타연료', '등유', '알코올', 'LPG(액화석유가스)',
       'CNG(압축천연가스)', '휘발유 하이브리드', 'LPG 하이브리드', '전기', 'LNG(액화천연가스)',
       '경유 하이브리드', 'CNG 하이브리드', '수소', '태양열'], dtype=object)

In [ ]:
# about 22.9s
smig2.loc[(smig2['연료'] == '휘발유') | (smig2['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
smig2.loc[(smig2['연료'] == '경유') | (smig2['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
smig2.loc[(smig2['연료'] == 'LPG(액화석유가스)') | (smig2['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
smig2.loc[(smig2['연료'] == 'CNG(압축천연가스)') | (smig2['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
smig2.loc[(smig2['연료'] == 'LNG(액화천연가스)') | (smig2['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
smig2.loc[smig2['연료'] == '전기', 'fuel'] = '전기'
smig2.loc[smig2['연료'] == '수소', 'fuel'] = '수소'
smig2.loc[smig2['연료'] == '태양열', 'fuel'] = '태양열'
smig2.loc[smig2['연료'] == '알코올', 'fuel'] = '알코올'
smig2.loc[smig2['연료'] == '등유', 'fuel'] = '등유'
smig2.loc[smig2['연료'] == '기타연료', 'fuel'] = '기타연료'
smig2.loc[smig2['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
smig2['fuel'].value_counts(dropna=False)

fuel
휘발유     13272774
경유       9926769
LPG      1918540
전기        368693
NaN       146798
CNG        30765
수소         28227
기타연료         933
LNG           27
알코올            3
태양열            2
등유             1
Name: count, dtype: int64

In [ ]:
smig2.shape

(25693532, 14)

In [ ]:
smig2['fuel'].value_counts(dropna=False).sum()

25693532

## 중복 차대번호 제거
- 최초등록일자 최신인 데이터 남기기

In [ ]:
# 1,020대 감소
smig2.shape[0], len(smig2['차대번호'].unique())

(25693532, 25692512)

In [ ]:
smig2['최초등록일자'].describe()

count   25693532.00
mean    20141515.50
std        61276.12
min     19501231.00
25%     20110228.00
50%     20150817.00
75%     20190408.00
max     20221227.00
Name: 최초등록일자, dtype: float64

In [ ]:
smig2.sort_values('최초등록일자', ascending=False)['최초등록일자'].head()

25693531    20221227
25672286    20221227
25672262    20221227
25672261    20221227
25672260    20221227
Name: 최초등록일자, dtype: int64

In [ ]:
# 36.4s
smig3 = smig2.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
smig3.shape

(25692512, 14)

## 차령 정보 계산 및 데이터 추가

In [ ]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [ ]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel'],
      dtype='object')

In [ ]:
smig3['차령'] = current_yr - smig3['차량연식']
smig3['차령'].describe()

count   25692512.00
mean           8.55
std            6.21
min            0.00
25%            4.00
50%            7.00
75%           12.00
max          123.00
Name: 차령, dtype: float64

In [ ]:
smig3['차령'].isnull().sum()

0

In [ ]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령'],
      dtype='object')

## 법정동코드로 지역정보 부착
- 시군구명 앞지역명만 추출

In [ ]:
smig3.shape

(25692512, 15)

In [ ]:
smig4 = smig3.merge(code, on='법정동코드', how='left')
smig4.shape

(25692512, 17)

In [ ]:
# 25.9s
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

0     고양시 일산서구
9      창원시 의창구
15     전주시 완산구
18     용인시 기흥구
30     청주시 상당구
Name: 시구군, dtype: object

In [ ]:
smig4.loc[smig4['시구군'].str.contains('부산') == True, '시구군'].head()

Series([], Name: 시구군, dtype: object)

In [ ]:
# 58.3s
smig4['시구군'] = smig4['시구군'].str.split(' ').str[0]
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

24226351    NaN
25154486    NaN
25315223    NaN
25341440    NaN
25345354    NaN
Name: 시구군, dtype: object

In [ ]:
smig4['시구군'].head()

0    고양시
1    연수구
2     진구
3    제주시
4    제주시
Name: 시구군, dtype: object

## 1일평균주행거리 정보 추가
- 매칭 컬럼(등록&제원|kosis)
- 1\. 시도, 시구군, 차종 | 시도명, 시군구명, 차종
- 2\. 시도, 차종 | 시도명, 차종
- 3\. 시도, 시구군, 차종(nan) | 시도명, 시군구명, 차종(합계)
    - 차종:nan인 데이터는 kosis의 차종:합계의 일일평균주행거리로 매칭

In [ ]:
smig4.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [ ]:
kos.columns

Index(['시도명', '시군구명', '차종', '전체'], dtype='object')

In [ ]:
smig4.shape

(25692512, 17)

In [ ]:
# about 9.1s
# 시도, 시구군, 차종으로 매칭
smig5 = smig4.merge(kos, left_on=['시도', '시구군', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
smig5.shape

(25692512, 20)

In [ ]:
smig5['전체'].isnull().sum()

341950

In [ ]:
# 4.8s
smig5y = smig5.loc[smig5['전체'].isnull() == False]
smig5n = smig5.loc[smig5['전체'].isnull() == True]
smig5y.shape, smig5n.shape

((25350562, 20), (341950, 20))

In [ ]:
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
smig5n = smig5n.drop(['시도명', '시군구명', '전체'], axis=1)
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [ ]:
smig5n.shape

(341950, 17)

In [ ]:
kos.shape

(1230, 4)

In [ ]:
kos.loc[kos['시군구명'] == '소계'].shape

(85, 4)

In [ ]:
# 시도, 차종으로만 매칭
smig5nm = smig5n.merge(kos.loc[kos['시군구명'] == '소계'].reset_index(drop=True), left_on=['시도', '차종'], right_on=['시도명', '차종'], how='left')
smig5nm.shape

(341950, 20)

In [ ]:
smig5nm.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
smig5nm['전체'].isnull().sum()

14

In [ ]:
smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']]

,시도,시구군,차종
341013,대구광역시,달서구,NaN
341057,대구광역시,남구,NaN
341078,서울특별시,구로구,NaN
341326,대구광역시,서구,NaN
341353,경상북도,경주시,NaN
341367,경상남도,창원시,NaN
341451,대구광역시,달성군,NaN
341452,전라남도,순천시,NaN
341489,경상남도,진주시,NaN
341569,울산광역시,북구,NaN


In [ ]:
# 차종이 빈값인 데이터는 kosis의 차종이 합계인 데이터로 매칭
for a, b, c in smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']].values:
    smig5nm.loc[(smig5nm['전체'].isnull() == True) & (smig5nm['시도'] == a) & ((smig5nm['시구군'] == b) & ((smig5nm['차종'].isnull() == True))), '전체'] = kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

In [ ]:
kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

35.7

In [ ]:
smig5nm['전체'].isnull().sum()

0

In [ ]:
# 6.4s
smig5 = pd.concat([smig5y, smig5nm], ignore_index=True)
smig5.shape

(25692512, 20)

In [ ]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
# 12.1s
smig5 = smig5.rename(columns={'전체':'일일평균주행거리'})
smig5 = smig5.drop(['시도명', '시군구명'], axis=1)
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [ ]:
smig5['일일평균주행거리'].isnull().sum()

0

## 저감장치 정보 추가

In [ ]:
att.columns

Index(['차대번호', '저감장치구분', '저감장치부착유무'], dtype='object')

In [ ]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [ ]:
smig5.shape

(25692512, 18)

In [ ]:
# 40.9s
smig6 = smig5.merge(att[['차대번호', '저감장치부착유무']], on='차대번호', how='left')
smig6.shape

(25692512, 19)

In [ ]:
smig6['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    25549321
유        143191
Name: count, dtype: int64

## [출력] 전처리 완료

In [ ]:
# 2m 47.1s
smig6.to_csv(os.path.join(ods_fold, '전처리완료_일일평균주행거리포함.csv'), index=False)

# [로드] 전처리

In [4]:
# 1m 18.3s
name = '전처리완료_일일평균주행거리포함'
file_name = f'{name}.csv'
smig6 = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
smig6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25692512 entries, 0 to 25692511
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   법정동코드     int64  
 1   차종        object 
 2   차대번호      object 
 3   최초등록일자    int64  
 4   차량연식      int64  
 5   배출가스인증번호  object 
 6   배출가스등급    object 
 7   차량말소YN    object 
 8   차종유형      object 
 9   연료        object 
 10  총중량       float64
 11  배기량       float64
 12  등급        object 
 13  fuel      object 
 14  차령        int64  
 15  시도        object 
 16  시구군       object 
 17  일일평균주행거리  float64
 18  저감장치부착유무  object 
dtypes: float64(3), int64(4), object(12)
memory usage: 3.6+ GB


## 3등급 경유차 DPF부착여부 수정
- 유

In [5]:
smig6['등급'].unique()

array(['X', '1', '2', '3', '4', '5'], dtype=object)

In [6]:
smig6['fuel'].unique()

array(['휘발유', 'LPG', '경유', '전기', nan, 'CNG', '수소', '기타연료', 'LNG', '태양열',
       '알코올', '등유'], dtype=object)

In [7]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'].value_counts(dropna=False)

NaN    7618185
유            3
Name: 저감장치부착유무, dtype: int64

In [8]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'] = '유'

In [9]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'].value_counts(dropna=False)

유    7618188
Name: 저감장치부착유무, dtype: int64

## result 4등급 파일 참고하여 DPF유무 수정

In [10]:
rdf = smig6.copy()
rdf.shape

(25692512, 19)

In [11]:
rdf_rest = rdf[rdf['등급'] != '4']
rdf4 = rdf[rdf['등급'] == '4']
rdf_rest.shape, rdf4.shape

((24181368, 19), (1511144, 19))

In [18]:
len(set(rdf4['차대번호'].unique()) - set(rs['차대번호'].unique()))

48589

In [19]:
rdf4['저감장치부착유무'].value_counts(dropna=False)

NaN    1511110
유           34
Name: 저감장치부착유무, dtype: int64

In [20]:
rs['DPF유무_수정'].value_counts(dropna=False)

무       879481
NaN     348422
유       261897
확인불가     12435
Name: DPF유무_수정, dtype: int64

In [21]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [22]:
rdf41 = rdf4.merge(rs, on='차대번호', how='left')

In [23]:
rdf41.loc[(rdf41['저감장치부착유무'] == '유') | (rdf41['DPF유무_수정'] == '유'), '저감장치부착유무'] = '유'

In [24]:
rdf41['저감장치부착유무'].value_counts(dropna=False)

NaN    1249231
유       261913
Name: 저감장치부착유무, dtype: int64

In [25]:
rdf41.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'DPF유무_수정'],
      dtype='object')

In [26]:
rdf4 = rdf41.drop('DPF유무_수정', axis=1)

In [27]:
smig6 = pd.concat([rdf_rest, rdf4], ignore_index=True)

In [28]:
smig6.shape

(25692512, 19)

## 연료로 분류

In [29]:
# 10.2s
# 휘발유, 경유, LPG
dgl = smig6.loc[(smig6['fuel'] == '휘발유') | (smig6['fuel'] == '경유') | (smig6['fuel'] == 'LPG')].reset_index(drop=True)
dgl.shape

(25117098, 19)

In [30]:
# 4.6s
# 나머지 : 전기, 수소, 태양열, CNG, LNG, 등유, 알코올, 기타연료, nan
etc = smig6[(smig6['fuel'] != '휘발유') & (smig6['fuel'] != '경유') & (smig6['fuel'] != 'LPG')].reset_index(drop=True)
etc.shape

(575414, 19)

In [31]:
dgl.shape[0] + etc.shape[0]

25692512

# 배출량 계산

## 자동차-엔진 가열(Hot-start) 배출

### 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

In [32]:
# 2.6s
df2 = dgl.copy()

In [33]:
df2['VKT'] = df2['일일평균주행거리'] * 365

In [34]:
df2['VKT'].isnull().sum()

0

### DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [35]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [36]:
DF_col = ['fuel', '차종', '차종유형', '차량연식', '차령']

In [37]:
df2[DF_col].head()

,fuel,차종,차종유형,차량연식,차령
0,휘발유,승용,중형,2023,0
1,휘발유,승용,중형,2023,0
2,휘발유,승용,대형,2023,0
3,휘발유,승용,대형,2023,0
4,휘발유,승용,대형,2023,0


In [38]:
df2[DF_col].values[0]

array(['휘발유', '승용', '중형', 2023, 0], dtype=object)

#### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [39]:
# about 26.6s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최이사님 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최이사님 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 25117098/25117098 [00:24<00:00, 1008128.11it/s]


In [40]:
df2.shape[0]

25117098

In [41]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(25117098, 25117098, 25117098, 25117098)

In [42]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [43]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- 저감장치부착유무 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [44]:
df2.loc[(df2['fuel'] == '경유') & (df2['저감장치부착유무'] == '유'), ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6

In [45]:
df2.shape[0]

25117098

In [46]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [47]:
df2[check_R_col].notnull().sum()

R_CO    8023247
R_HC    8023247
R_PM    8023247
dtype: int64

In [48]:
df2.loc[df2['저감장치부착유무'] == '유', DF_col + check_DF_col + check_R_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
26,경유,화물,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
28,경유,화물,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
60,경유,화물,소형,2022,1,1.00,1.00,1.00,1.00,99.50,90.00,83.60
90,경유,승용,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
91,경유,승용,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60


In [49]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [50]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

### ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최이사님)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [51]:
EFi_col = ['차종', '차종유형', 'fuel', '차량연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [52]:
df2[EFi_col].values[0]

array(['승용', '중형', '휘발유', 2023], dtype=object)

In [53]:
df2['차종유형'].unique()

array(['중형', '대형', '경형', '소형', nan], dtype=object)

In [54]:
# about 2m 10s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
EFi_NH3_list = []

# 차종, 차종유형, 연료, 연식
for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    EFi_COm = 0
    EFi_HCm = 0
    EFi_NOxm = 0
    EFi_PM10m = 0
    EFi_PM2_5m = 0
    EFi_NH3m = 0
    for V in [30, 70, 100]:
        if fuel == '휘발유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 8.6275 * (V**-1.0722)
                    EFi_NOx = 3.2294 * (V**-0.5763)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 5.1835 * (V**-1.1889)
                    EFi_NOx = 1.7525 * (V**-0.6481)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 0.7446 * (V**-0.9392)
                    EFi_NOx = 0.3403 * (V**-0.5455)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.9952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 45:
                        EFi_CO = 4.5956 * (V**-0.8461)
                    else:
                        EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                    EFi_HC = 0.2662 * (V**-0.7830)
                    EFi_NOx = 0.4476 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 45:
                        EFi_CO = 4.4517 * (V**-0.8461)
                    else:
                        EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                    EFi_HC = 0.2556 * (V**-0.7830)
                    EFi_NOx = 0.4353 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 45:
                        EFi_CO = 4.3079 * (V**-0.8461)
                    else:
                        EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                    EFi_HC = 0.2449 * (V**-0.7830)
                    EFi_NOx = 0.4230 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.164 * (V**-0.8461)
                    else:
                        EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                    EFi_HC = 0.2343 * (V**-0.7830)
                    EFi_NOx = 0.4106 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 111.67 * (V**-1.1566)
                    EFi_HC = 32.017 * (V**-1.4171)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 22.356 * (V**-0.9068)
                    EFi_HC = 0.8428 * (V**-0.8829)
                    EFi_NOx = 1.2613 * (V**-0.3873)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 1.4898 * (V**-0.3837)
                    EFi_HC = 0.1738 * (V**-0.7268)
                    EFi_NOx = 0.1563 * (V**-0.2671)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 65.4:
                        EFi_CO = 1.4082 * (V**-0.7728)
                    else:
                        EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 1.2955 * (V**-0.7728)
                    else:
                        EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 65.4:
                        EFi_CO = 1.2550 * (V**-0.7728)
                    else:
                        EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 65.4:
                        EFi_CO = 1.2144 * (V**-0.7728)
                    else:
                        EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 1.1739 * (V**-0.7728)
                    else:
                        EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 11.249 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 16.965 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                    EFi_HC = 3.0285 * (V**-0.7830)
                    EFi_NOx = 1.9923 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 1.8928 * (V**-0.7830)
                    EFi_NOx = 1.2352 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 0.9227 * (V**-0.7830)
                    EFi_NOx = 3.8859 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 2000:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2001 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 11.920 * (V**-1.5356)
                    EFi_NOx = 3.0649 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 37.573 * (V**-1.2078)
                    EFi_HC = 3.1786 * (V**-1.5356)
                    EFi_NOx = 1.4931 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.08
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '화물') and (car_size == '소형') ):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 36.578 * (V**-1.2078)
                    EFi_HC = 3.0337 * (V**-1.5356)
                    EFi_NOx = 2.0104 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '중형'):
                if 1993 <= car_birth <= 1998:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 1999:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2008: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '대형'):
                if 1993 <= car_birth:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                else:
                    EFi_CO = 1
                    EFi_HC = 1
                    EFi_NOx = 1
                    EFi_PM10 = 1
                    EFi_PM2_5 = 1
                EFi_NH3 = 0.002
        elif fuel == '경유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)      # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)     # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)        # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)   # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)           # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '중형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.6930 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5414 * (V**-0.7524)
                    EFi_HC = 0.0927 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0396 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)            # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.9609 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.7507 * (V**-0.7524)
                    EFi_HC = 0.1554 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0361 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 0.9835 * (V**-0.5096)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 1.1412 * (V**-0.4324)
                    EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
                elif 1991 <= car_birth <= 1995:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.6313 * (V**-0.7298)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5999 * (V**-0.3294)
                    EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
                elif 1996 <= car_birth <= 1997:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6408 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5168 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2894 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    if V <= 65.4:
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_NOx = 0.0271 * (V**0.7596)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 32.550 * (V**-0.4944)
                    EFi_HC = 15.753 * (V**-0.5912)
                    if V < 80:
                        EFi_NOx = 40.692 * (V**-0.5590)
                    else:
                        EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                    EFi_PM10 = 5.4886 * (V**-0.5911)
                    EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 22.804 * (V**-0.4660)
                    else:
                        EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 25.708 * (V**-0.4772)
                    else:
                        EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 16.378 * (V**-0.5340)
                    EFi_HC = 5.8477 * (V**-0.5466)
                    if V < 80:
                        EFi_NOx = 25.436 * (V**-0.4656)
                    else:
                        EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                    EFi_PM10 = 1.2848 * (V**-0.4715)
                    EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    EFi_NOx = 15.001 * (V**-0.4528)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                        EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                        EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 28.205 * (V**-0.5337)
                    EFi_HC = 6.1146 * (V**-0.4979)
                    EFi_NOx = 41.346 * (V**-0.3645)
                    EFi_PM10 = 5.2158 * (V**-0.5048)
                    EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 2.4911 * (V**-0.4149)
                    EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 1.4432 * (V**-0.3870)
                    EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 0.9375 * (V**-0.3910)
                    EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 1.1507 * (V**-0.4804)
                    EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 9.6452 * (V**-0.5291)
                    EFi_HC = 3.2339 * (V**-0.7436)
                    EFi_NOx = 30.5870 * (V**-0.3548)
                    if V <= 80:
                        EFi_PM10 = 0.4657 * (V**-0.5634)
                        EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                    else:
                        EFi_PM10 = 0.0014 * (V**0.7970)
                        EFi_PM2_5 = k * 0.0014 * (V**0.7970)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.003
                elif 2015 <= car_birth:
                    EFi_CO = 11.4415 * (V**-0.8036)
                    EFi_HC = 0.6774 * (V**-0.8321)
                    EFi_NOx = 112.1229 * (V**-1.6393)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 4.5854 * (V**-0.3613)
                    EFi_HC = 0.4840 * (V**-0.2756)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.8117 * (V**-0.4071)
                    EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
                elif 1991 <= car_birth <= 1995:
                    EFi_CO = 3.4774 * (V**-0.3483)
                    EFi_HC = 0.4844 * (V**-0.3288)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6188 * (V**-0.4540)
                    EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 3.3934 * (V**-0.3837)
                    EFi_HC = 0.4955 * (V**-0.3393)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.7037 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6157 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.4838 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth <= 2016:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                        EFi_NOx = 0.0271 * (V**0.7596)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '화물') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)     
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif car_type == '특수':
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2009:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.003
                elif car_birth == 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 30.402 * (V**-0.4685)
                    EFi_HC = 15.75 * (V**-0.582)
                    EFi_NOx = 117.49 * (V**-0.365)
                    EFi_PM10 = 7.6212 * (V**-0.4183)
                    EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 6.0264 * (V**-0.4627)
                    EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 4.873 * (V**-0.4382)
                    EFi_PM2_5 = k * 4.873 * (V**-0.4382)
                    EFi_NH3 = 0.003
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 28.399 * (V**-0.5999)
                    EFi_HC = 10.031 * (V**-0.5828)
                    EFi_NOx = 85.301 * (V**-0.4023)
                    EFi_PM10 = 3.7541 * (V**-0.4055)
                    EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 52.136 * (V**-0.8618)
                    EFi_HC = 3.7878 * (V**-0.5425)
                    EFi_NOx = 107.5 * (V**-0.5679)
                    EFi_PM10 = 2.6847 * (V**-0.6112)
                    EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
                    EFi_NH3 = 0.003
                elif car_birth == 2008:                    # 2008 <= car_birth <= 2009 -> 2008 == car_birth: 변경 (⭕)
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2009 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 4.3762 * (V**-0.4550)
                    EFi_HC = 0.3627 * (V**-0.7071)
                    EFi_NOx = 18.0405 * (V**-1.0986)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
        elif fuel == 'LPG':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 101.79 * (V**-1.6823)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 7.5371 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 3.2821 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 39.362 * (V**-1.0085)
                    EFi_HC = 2.8981 * (V**-1.3927)
                    EFi_NOx = 1.8419 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 73.022 * (V**-1.2078)
                    EFi_HC = 4.4166 * (V**-1.5356)
                    EFi_NOx = 2.0280 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth == 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 6.1701 * (V**-1.0719)
                        EFi_HC = 2.3782 * (V**-1.9979)
                    else:
                        EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                        EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                    if V <= 97.3:
                        EFi_NOx = 0.4758 * (V**-1.0665)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            # 최이사님 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
            elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 16.491 * (V**-1.4713)
                        EFi_HC = 1.9218 * (V**-1.8418)
                    else:
                        EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                        EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                    if V <= 97.3:
                        EFi_NOx = 0.1246 * (V**-0.441)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
        # else:
        #     # 값 고민 : 1
        #     # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        #     EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

        if V < 60:
            EFi_COm += EFi_CO*0.34
            EFi_HCm += EFi_HC*0.34
            EFi_NOxm += EFi_NOx*0.34
            EFi_PM10m += EFi_PM10*0.34
            EFi_PM2_5m += EFi_PM2_5*0.34
            EFi_NH3m += EFi_NH3*0.34
        elif 60 <= V < 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33
        elif V > 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_COm)
    EFi_HC_list.append(EFi_HCm)
    EFi_NOx_list.append(EFi_NOxm)
    EFi_PM10_list.append(EFi_PM10m)
    EFi_PM2_5_list.append(EFi_PM2_5m)
    EFi_NH3_list.append(EFi_NH3m)

100%|██████████| 25117098/25117098 [02:10<00:00, 192108.12it/s]


In [55]:
df2.shape[0]

25117098

In [56]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list), len(EFi_NH3_list)

(25117098, 25117098, 25117098, 25117098, 25117098, 25117098)

In [57]:
# 28.1s
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list
df2['EFi_NH3'] = EFi_NH3_list

In [58]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

#### 등급별 확인

In [59]:
grp =  df2.groupby('등급').agg({'EFi_CO':'sum', 'EFi_HC':'sum', 'EFi_NOx':'sum', 'EFi_PM10':'sum', 'EFi_PM2_5':'sum', '차대번호':'count'})
grp = grp.rename(columns={'차대번호':'차량대수'})
grp['EFi_CO_per'] = grp['EFi_CO'] / grp['차량대수']
grp['EFi_HC_per'] = grp['EFi_HC'] / grp['차량대수']
grp['EFi_NOx_per'] = grp['EFi_NOx'] / grp['차량대수']
grp['EFi_PM10_per'] = grp['EFi_PM10'] / grp['차량대수']
grp['EFi_PM2_5_per'] = grp['EFi_PM2_5'] / grp['차량대수']
grp

,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,차량대수,EFi_CO_per,EFi_HC_per,EFi_NOx_per,EFi_PM10_per,EFi_PM2_5_per
등급,,,,,,,,,,,
1,172144.48,3634.71,34114.45,994.54,914.97,2376172,0.07,0.00,0.01,0.00,0.00
2,1616751.27,39277.66,162635.72,4977.85,4580.94,11182913,0.14,0.00,0.01,0.00,0.00
3,1497885.60,117982.08,5679344.80,25704.45,23566.80,8753079,0.17,0.01,0.65,0.00,0.00
4,618366.57,77648.44,2237614.63,22953.20,21059.76,1511032,0.41,0.05,1.48,0.02,0.01
5,1473874.92,319308.99,3856831.26,164470.38,155975.51,1154641,1.28,0.28,3.34,0.14,0.14
X,23078.11,1417.72,20095.88,368.83,348.51,139261,0.17,0.01,0.14,0.00,0.00


### 배출량 계산

In [60]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.06,0.00,0.01,0.00,0.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.06,0.00,0.01,0.00,0.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00


In [61]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)

In [62]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [63]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.06,0.00,0.01,0.00,0.00,0.75,0.02,0.16,0.01,0.01
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.06,0.00,0.01,0.00,0.00,0.77,0.02,0.17,0.01,0.01
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.34,0.02,0.21,0.01,0.01
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.34,0.02,0.21,0.01,0.01
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.32,0.02,0.21,0.01,0.01


In [64]:
# 소수점 옵션
pd.options.display.float_format = '{:.5f}'.format
# pd.reset_option('display.float_format')

In [65]:
# 속력 : 75
grp =  df2.groupby('등급').agg({'E_HOT_CO':'sum', 'E_HOT_HC':'sum', 'E_HOT_NOx':'sum', 'E_HOT_PM10':'sum', 'E_HOT_PM2_5':'sum', '차대번호':'count'})
grp = grp.rename(columns={'차대번호':'차량대수'})
grp['E_HOT_CO_per'] = grp['E_HOT_CO'] / grp['차량대수']
grp['E_HOT_HC_per'] = grp['E_HOT_HC'] / grp['차량대수']
grp['E_HOT_NOx_per'] = grp['E_HOT_NOx'] / grp['차량대수']
grp['E_HOT_PM10_per'] = grp['E_HOT_PM10'] / grp['차량대수']
grp['E_HOT_PM2_5_per'] = grp['E_HOT_PM2_5'] / grp['차량대수']
grp

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
등급,,,,,,,,,,,
1,2376503.40452,50142.55093,470417.32384,13703.06074,12606.81588,2376172,1.00014,0.02110,0.19797,0.00577,0.00531
2,25678957.23571,671974.33470,2662401.21633,77198.89063,71076.20922,11182913,2.29627,0.06009,0.23808,0.00690,0.00636
3,13179760.29428,728140.09287,96367096.33584,94680.03621,86774.52991,8753079,1.50573,0.08319,11.00951,0.01082,0.00991
4,16110917.71482,1938119.41480,45356115.31767,492724.85531,452035.14794,1511032,10.66219,1.28265,30.01665,0.32608,0.29916
5,35917147.68151,7724001.74905,82477058.35623,3799310.86813,3589668.34840,1154641,31.10677,6.68953,71.43091,3.29047,3.10890
X,454764.93735,34223.96330,353826.78142,8716.28144,8211.53437,139261,3.26556,0.24575,2.54075,0.06259,0.05897


## 자동차-엔진 미가열(Cold-start) 배출

### 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

### 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [66]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [67]:
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [68]:
df2['Beta(Estimated)'].head()

0   0.27643
1   0.27643
2   0.27643
3   0.27643
4   0.27643
Name: Beta(Estimated), dtype: float64

In [69]:
df2['Beta(Estimated)'].isnull().sum()

0

### $e^{COLD} / e^{HOT}$

In [70]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [71]:
df2['차량연식'].describe()

count   25117098.00000
mean        2014.35604
std            6.18721
min         1900.00000
25%         2011.00000
50%         2015.00000
75%         2019.00000
max         2023.00000
Name: 차량연식, dtype: float64

In [72]:
# 1m 31.7s
# 휘발유(자동 제어)
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['fuel'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [73]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     4282101
eCOLD/eHOT_NOx    4282101
eCOLD/eHOT_HC     4282101
eCOLD/eHOT_PM     4248868
dtype: int64

In [74]:
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [75]:
df2.loc[df2['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [76]:
df2.loc[df2['eCOLD/eHOT_NOx'] < 0, 'eCOLD/eHOT_NOx'].shape

(0,)

In [77]:
df2.loc[df2['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [78]:
df2.loc[df2['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

### 배출량 계산

In [79]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

#### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [80]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)

In [81]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].shape

(1758697, 6)

In [82]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].head()

,fuel,Beta(Estimated),VKT,EFi_NOx,eCOLD/eHOT_NOx,E_COLD_NOx
5,LPG,0.27643,12811.50000,0.02101,0.90560,-7.02283
6,LPG,0.27643,13578.00000,0.02101,0.90560,-7.44300
7,LPG,0.27643,12775.00000,0.02101,0.90560,-7.00282
8,LPG,0.27643,13906.50000,0.02101,0.90560,-7.62307
9,LPG,0.27643,14855.50000,0.02101,0.90560,-8.14328


In [83]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df2[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,0.75004,0.01696,0.16247,0.00578,0.00532,1435.58832,116.12478,50.84205,0.00000,0.00000
1,0.76704,0.01734,0.16615,0.00591,0.00544,1468.12290,118.75651,51.99428,0.00000,0.00000
2,1.34118,0.02229,0.21356,0.00760,0.00699,2567.05604,152.63994,66.82921,0.00000,0.00000
3,1.34118,0.02229,0.21356,0.00760,0.00699,2567.05604,152.63994,66.82921,0.00000,0.00000
4,1.31806,0.02191,0.20988,0.00746,0.00687,2522.79645,150.00822,65.67698,0.00000,0.00000


In [84]:
df2[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
25117093,47.27213,6.86754,11.08525,0.01198,0.01102,10349.51731,100.50995,1002.36274,0.00000,0.00000
25117094,47.27213,6.86754,11.08525,0.01198,0.01102,10349.51731,100.50995,1002.36274,0.00000,0.00000
25117095,47.27213,6.86754,11.08525,0.01198,0.01102,10349.51731,100.50995,1002.36274,0.00000,0.00000
25117096,47.27213,6.86754,11.08525,0.01198,0.01102,10349.51731,100.50995,1002.36274,0.00000,0.00000
25117097,47.27213,6.86754,11.08525,0.01198,0.01102,10349.51731,100.50995,1002.36274,0.00000,0.00000


In [85]:
df2[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

#### 대형 차량 E_COLD 값 제거

In [86]:
df2.loc[df2['차종유형'] == '대형', ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']] = [0, 0, 0, 0, 0]

In [87]:
df2.loc[df2['차종유형'] == '대형', ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']].head()

,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
2,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000
15,0.00000,0.00000,0.00000,0.00000,0.00000
16,0.00000,0.00000,0.00000,0.00000,0.00000


## 자동차-휘발유 증발 배출

### 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

### 배출계수

#### 🔺 수정 중

#### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

##### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [88]:
RVP = 65

##### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)
    - 기상자료개방포털(2022년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 8°C(최저), 12.9°C(평균), 18.6°C(최고)

In [89]:
t_min = 8 # 평균 최저 온도
t_max = 18.6
t_a = (t_min + t_max) / 2
t_rise = 10.6 # 평균 상승 온도

In [90]:
df2.loc[df2['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [91]:
df2['e_d'].isnull().sum()

0

#### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [92]:
df2.loc[df2['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [93]:
df2['e_RHOT'].isnull().sum()

0

#### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [94]:
df2.loc[df2['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [95]:
df2.loc[df2['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [96]:
df2.loc[df2['차량연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['차량연식'] >= 1991, 'e_fi'] = 0

#### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [97]:
df2.loc[df2['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [98]:
df2['e_RWARM'].isnull().sum()

0

### $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

#### ❓ 확인 중
- 수정 사항(2023.04.20, 최이사님)
    - 찾는 중
- 수정 사항(2023.04.20, 최이사님)
    - w:p = 0.8:0.2

In [99]:
p = 0.2 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.8 # 미가열 엔진 상태로 끝나는 trip의 비율

In [100]:
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [101]:
df2['R'].isnull().sum()

0

### $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [102]:
# 8.9s
not_injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991)].shape[0]
injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991)].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.9931614094087922

In [103]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

### $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [104]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * ( p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'] )

### $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [105]:
df2['E_EVA_VOC'] = 365 * (df2['e_d'] + df2['S_c'] + df2['S_fi']) + df2['R']

In [106]:
df2['E_EVA_VOC'].isnull().sum()

0

In [107]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [108]:
# 휘발유만 증발 배출 존재
df2.loc[(df2['fuel'] == 'LPG') | (df2['fuel'] == '경유'), 'E_EVA_VOC'] = 0

## 배출량 합계
$E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$

### issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [109]:
df2[df2['E_COLD_NOx'] < 0].shape

(1413432, 58)

In [110]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [111]:
df2[df2['E_COLD_NOx'] < 0].shape

(0, 58)

In [112]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [113]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['E_COLD_PM10'] != 0)].shape

(0, 58)

In [114]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [115]:
# 단위 변환(g -> kg)
df2['E_COLD_CO'] = df2['E_COLD_CO'] / 1000
df2['E_COLD_HC'] = df2['E_COLD_HC'] / 1000
df2['E_COLD_NOx'] = df2['E_COLD_NOx'] / 1000
df2['E_COLD_PM10'] = df2['E_COLD_PM10'] / 1000
df2['E_COLD_PM2_5'] = df2['E_COLD_PM2_5'] / 1000
df2['E_EVA_VOC'] = df2['E_EVA_VOC'] / 1000

In [116]:
# 26.5s
fuel = '휘발유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC'] + df2.loc[df2['fuel'] == fuel , 'E_EVA_VOC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [117]:
# 22.7s
fuel = '경유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [118]:
# 20.0s
fuel = 'LPG'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [119]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df2[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,2.18563,0.41662,0.27860,0.00578,0.00532
1,2.23516,0.41956,0.28491,0.00591,0.00544
2,1.34118,0.39060,0.21356,0.00760,0.00699
3,1.34118,0.39060,0.21356,0.00760,0.00699
4,1.31806,0.38881,0.20988,0.00746,0.00687


## 황산화물 배출량

In [120]:
# 42.4s
fcc_list = []
for fone, cd, ty in tqdm(df2[['fuel', '차종', '차종유형']].values):
    fccm = 0
    for V in [30, 70, 100]:
        if fone == '휘발유':
            if cd == '승용':
                fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
            elif cd == '승합':
                if ty =='경형':
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '화물':
                if ty == '경형':   
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '특수':
                if ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
        elif fone == '경유':
            if cd == '승용':
                fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
            elif cd == '승합':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1425.2 * (V**(-0.7593)) # E17
                    else:
                        fcc = 0.0082 * (V**2) - 0.0430*V + 60.12 # E17
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1919.0 * (V**(-0.5396)) # E16
                    else:
                        fcc = 0.0447 * (V**2) - 7.072*V + 478 # E16
            elif cd == '화물':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19
            elif cd == '특수':
                if ty == '소형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                if ty == '중형' or ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19
        elif fone == 'LPG':
            if cd == '승용':
                fcc = 45 # E09
            elif cd == '승합':
                fcc = 45 # E09
            elif cd == '화물':
                if ty == '경형' or ty == '소형' or ty == '중형':
                    fcc = 45 # E09
            elif cd == '특수':
                if ty == '소형' or ty == '중형'or ty == '대형':
                    fcc = 45 # E09
        if V < 60:
            fccm += fcc * 0.34
        elif 60 <= V < 90:
            fccm += fcc * 0.33
        elif V > 90:
            fccm += fcc * 0.33
    fcc_list.append(fccm)

100%|██████████| 25117098/25117098 [00:41<00:00, 598810.82it/s]


In [121]:
df2.shape

(25117098, 63)

In [122]:
len(fcc_list)

25117098

In [123]:
# 5.0s
# 단위 : %
sf_list = []
for fone in tqdm(df2['fuel']):
    if fone == '경유':
        sf = 0.017
    elif fone == '휘발유':
        sf = 0.001
    elif fone == 'LPG':
        sf = 0.015
    sf_list.append(sf)

  2%|▏         | 384754/25117098 [00:00<00:06, 3828103.33it/s]

100%|██████████| 25117098/25117098 [00:05<00:00, 4380452.36it/s]


In [124]:
df2.shape, len(sf_list)

((25117098, 63), 25117098)

In [125]:
# 3.5s
df2['fcc'] = fcc_list
df2['sf'] = sf_list

In [126]:
df2['E_SOx_total'] = df2['VKT'] * df2['fcc'] * df2['sf'] * 2 / (1000*100)

## NH3

In [127]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total'],
      dtype='object')

In [128]:
df2['VKT'].describe()

count   25117098.00000
mean       14437.04352
std         3300.01418
min         2847.00000
25%        13030.50000
50%        13505.00000
75%        14271.50000
max        98185.00000
Name: VKT, dtype: float64

In [129]:
df2['EFi_NH3'].isnull().sum()

0

In [130]:
df2['EFi_NH3'].describe()

count   25117098.00000
mean           0.01045
std            0.01488
min            0.00100
25%            0.00190
50%            0.01126
75%            0.01126
max            0.13435
Name: EFi_NH3, dtype: float64

In [131]:
df2['E_NH3_total'] = df2['VKT'] * df2['EFi_NH3'] / 1000

In [132]:
df2['E_NH3_total'].describe()

count   25117098.00000
mean           0.14367
std            0.20020
min            0.00675
25%            0.02510
50%            0.14590
75%            0.15659
max            5.29256
Name: E_NH3_total, dtype: float64

## CO2

### 계산식

In [133]:
# 57.7s
fc_list = []
for ty, yr, fuel, cc, twg in tqdm(df2[['차종유형', '차량연식', 'fuel', '배기량', '총중량']].values):
    fcm = 0
    for V in [30, 70, 100]:
        if fuel == '휘발유' and ty != '경형' and ty !='대형':
            if yr <= 1971:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 521 * V**(-0.554)
                    elif 60 <= V < 80:
                        fc = 55
                    elif 80 <= V <130:
                        fc = 0.386*V + 24143
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 681 * V**(-0.583)
                    elif 60 <= V < 80:
                        fc = 67
                    elif 80 <= V <130:
                        fc = 0.471*V + 29286
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 979 * V**(-0.628)
                    elif 60 <= V < 80:
                        fc = 80
                    elif 80 <= V <130:
                        fc = 0.414*V + 46867
            elif 1972 <= yr <= 1977:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 595 * V**(-0.63)
                    elif 60 <= V < 130:
                        fc = 95 - 1.324*V + 0.0086*V**2
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 864 * V**(-0.69)
                    elif 60 <= V < 130:
                        fc = 59 - 0.407*V + 0.0042*V**2
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 1236 * V**(-0.764)
                    elif 60 <= V < 130:
                        fc = 65 - 0.407*V + 0.0042*V**2
            elif 1978 <= yr <= 1985:
                if cc < 1400:
                    if 10 <= V < 50:
                        fc = 544 * V**(-0.63)
                    elif 50 <= V < 130:
                        fc = 85 - 1.108*V + 0.0077*V**2
                elif 1400 <= cc < 2000:
                    if 10 <= V < 50:
                        fc = 879 * V**(-0.72)
                    elif 50 <= V < 130:
                        fc = 71 - 0.7032*V + 0.0059*V**2
                elif cc >= 2000:
                    if 10 <= V < 50:
                        fc = 1224 * V**(-0.756)
                    elif 50 <= V < 130:
                        fc = 111 - 1.333*V + 0.0093*V**2
            elif 1986 <= yr <= 1992:
                if cc < 1400:
                    if 10 <= V < 17.9:
                        fc = 296.7 - 80.21*ln(V)
                    elif 17.9 <= V < 130:
                        fc = 81.1 - 1.014*V + 0.0068*V**2
                elif 1400 <= cc < 2000:
                    if 10 <= V < 22.3:
                        fc = 606.1 * V**(-0.667)
                    elif 22.3 <= V < 130:
                        fc = 102.5 - 1.364*V + 0.0086*V**2
                elif cc >= 2000:
                    if 10 <= V < 59.5:
                        fc = 819.9 * V**(-0.663)
                    elif 59.5 <= V < 130:
                        fc = 41.7 - 0.122*V + 0.0016*V**2
            elif yr > 1993:
                if cc < 1400:
                    if 10 <= V < 130:
                        fc = 85.55 - 1.383*V + 0.0117*V**2
                elif 1400 <= cc < 2000:
                    if 10 <= V < 130:
                        fc = 109.6 - 1.98*V + 0.0168*V**2
                else:
                    fc = 0
        elif fuel == '휘발유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.0167*V**2 - 2.649*V + 161.51
            else:
                fc = 0
        elif fuel == '휘발유' and ty == '대형':
            if twg > 3500:
                if V <= 60:
                    fc = 225
                elif 60 < V <= 90:
                    fc = 150
                elif V > 90:
                    fc = 165
            else:
                fc = 0
        elif fuel == '경유' and ty != '경형' and ty != '대형':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 91.106 - 1.308*V + 0.00871*V**2
            else:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 118.489 - 2.084*V + 0.014*V**2
                else:
                    fc = 0
        elif fuel == '경유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.02113*V**2 - 2.65*V + 148.91
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '경유' and ty == '대형':
            if twg > 3500:
                if twg < 7500:
                    if 0 <= V < 47:
                        fc = 1425.2*V**(-0.7593)
                    elif 47 <= V < 100:
                        fc = 0.0082*V**2 - 0.0430*V + 60.12
                    else:
                        fc = 0
                elif 7500 <= twg < 16000:
                    if 0 <= V < 59:
                        fc = 1068.4*V**(-0.4905)
                    elif 59 <= V < 100:
                        fc = 0.0126*V**2 - 0.6589*V + 141.18
                    else:
                        fc = 0
                elif 16000 <= twg < 32000:
                    if 0 <= V < 59:
                        fc = 1595.1*V**(-0.4744)
                    elif 59 <= V < 100:
                        fc = 0.0382*V**2 - 5.1630*V + 399.3
                    else:
                        fc = 0
                elif twg > 32000:
                    if 0 <= V < 58:
                        fc = 1855.7*V**(-0.4367)
                    elif 58 <= V < 100:
                        fc = 0.0765*V**2 - 11.414*V + 720.9
                    else:
                        fc = 0
        elif fuel == 'LPG':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 0.00720*V**2 - 0.9250*V + 74.625
                    else:
                        fc = 0
                else:
                    fc = 0
            else:
                if twg < 2500:
                    if V <= 60:
                        fc = 59
                    elif 60 < V <= 90:
                        fc = 45
                    else:
                        fc = 45
                else:
                    fc = 0
        else:
            fc = 0

        if V < 60:
            fcm += fc*0.34
        elif 60 <= V <= 90:
            fcm += fc*0.33
        elif V > 90:
            fcm += fc*0.33
    fc_list.append(fcm)

100%|██████████| 25117098/25117098 [00:51<00:00, 492092.65it/s]


In [134]:
df2.shape, len(fc_list)

((25117098, 67), 25117098)

In [135]:
df2['FC'] = fc_list

In [136]:
df2[df2['FC'] == 0].shape

(8442351, 68)

In [137]:
r_HC_list = []
for fuel in tqdm(df2['fuel']):
    if fuel == '휘발유':
        r_HC = 1.8
    elif fuel == '경유':
        r_HC = 2.0
    else:
        r_HC = 1.3 # chatGPT 최이사님(2023.06.22.목)
    r_HC_list.append(r_HC)

100%|██████████| 25117098/25117098 [00:05<00:00, 4432584.53it/s]


In [138]:
df2.shape, len(r_HC_list)

((25117098, 68), 25117098)

In [139]:
df2['r_HC'] = r_HC_list

In [140]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total', 'E_NH3_total', 'FC',
       'r_HC'],
      dtype='object')

In [141]:
df2['E_CO_total'].describe()

count   25117098.00000
mean           5.75527
std           24.46323
min            0.00063
25%            0.10625
50%            1.41310
75%            2.53319
max         3439.64320
Name: E_CO_total, dtype: float64

In [142]:
df2.loc[df2['E_CO_total'] > 3000, ['E_HOT_CO', 'E_COLD_CO', 'E_EVA_VOC', 'E_CO_total', '차종', '차종유형', 'VKT', '총중량', '배기량', '차량연식']]

,E_HOT_CO,E_COLD_CO,E_EVA_VOC,E_CO_total,차종,차종유형,VKT,총중량,배기량,차량연식
21362875,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3015.00000,4701.00000,2006
21395635,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3015.00000,4701.00000,2006
21491801,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3015.00000,4701.00000,2006
21963565,3018.46240,0.00000,0.40764,3018.46240,화물,중형,25112.00000,3060.00000,4701.00000,2004
22220546,3439.64320,0.00000,0.42450,3439.64320,화물,중형,28616.00000,3060.00000,4701.00000,2004
22300865,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3060.00000,4701.00000,2003
22329769,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3060.00000,4701.00000,2003
22338414,3018.46240,0.00000,0.40764,3018.46240,화물,중형,25112.00000,3060.00000,4701.00000,2003
22347269,3018.46240,0.00000,0.40764,3018.46240,화물,중형,25112.00000,3060.00000,4701.00000,2003
22410467,3044.78620,0.00000,0.40870,3044.78620,화물,중형,25331.00000,3060.00000,4701.00000,2003


In [143]:
df2['E_CO2_total'] = 44.011 * ( (df2['FC'] / 1000) / (12.011 + 1.008 * df2['r_HC'])) - df2['E_CO_total'] / 28.011 - df2['E_HC_total'] / 13.85 - (df2['E_PM10_total'] + df2['E_PM2_5_total']) / 12.011

In [144]:
df2['E_CO2_total'].describe()

count   25117098.00000
mean          -0.15491
std            1.19382
min         -142.21128
25%           -0.06998
50%            0.06369
75%            0.10530
max          146.79612
Name: E_CO2_total, dtype: float64

In [145]:
df2[df2['E_CO2_total'] < 0].shape

(11840265, 70)

### 음수 0으로 처리

In [146]:
df2.loc[df2['E_CO2_total'] < 0, 'E_CO2_total'] = 0

## [출력]

In [147]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230704'

In [148]:
# # 24m 55.9s
# df2.to_csv(os.path.join(df_fold, f'CAPSS전체결과({today_date}).csv'), index=False)

In [641]:
# f'CAPSS전체결과({today_date})'

# [로드] 계산 결과

In [642]:
# # 
# date = '20230629'
# name = f'CAPSS전체결과({date})'
# file_name = f'{name}.csv'
# df2 = pd.read_csv(os.path.join(df_fold, file_name), low_memory=False)
# df2.info()

# 통계

## 배출량 통계1
- 조건
    - 승용(경형, 소형, 중형)
    - 화물(소형, 중형)

In [643]:
ad = df2.loc[(df2['차종'] == '승용') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) | (df2['차종'] == '화물') & ((df2['차종유형'] == '소형') | (df2['차종유형'] == '중형'))].reset_index(drop=True)
ad.shape

(17575496, 70)

In [644]:
adg = ad[ad['fuel'] == '휘발유']
adl = ad[ad['fuel'] == 'LPG']
add = ad[ad['fuel'] == '경유']
ad.shape[0] == adg.shape[0] + adl.shape[0] + add.shape[0]

True

In [645]:
grp1 = adg.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,1476949.43335,380415.16688,60118.69784,3357.69582,3089.08015,4594.02121,98977.97313,0.00000,988804
2,24257957.42603,3029090.47473,482292.55028,27155.55442,24983.10452,34335.32476,814936.80582,13.67732,7402335
3,3159139.42506,296745.04894,447471.08798,3219.74863,2962.16874,2478.92356,158883.37520,0.00000,542026
4,4482352.14201,535121.07565,826299.34603,1937.35233,1782.36415,1343.66918,90649.90240,25.10767,291556
5,2667721.62654,417654.10191,222165.58099,232.53766,213.93465,137.27460,8222.79065,837.78269,28848
X,174593.19662,29120.59356,7173.71365,246.36038,226.65155,320.64329,7429.40781,725.29966,68388


In [646]:
grp1['CO'] = grp1['E_CO_total'] / grp1['차대번호']
grp1['NOx'] = grp1['E_NOx_total'] / grp1['차대번호']
grp1['SOx'] = grp1['E_SOx_total'] / grp1['차대번호']
grp1['PM10'] = grp1['E_PM10_total'] / grp1['차대번호']
grp1['PM2.5'] = grp1['E_PM2_5_total'] / grp1['차대번호']
grp1['HC'] = grp1['E_HC_total'] / grp1['차대번호']
grp1['NH3'] = grp1['E_NH3_total'] / grp1['차대번호']
grp1['CO2'] = grp1['E_CO2_total'] / grp1['차대번호']
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,1476949.43335,380415.16688,60118.69784,3357.69582,3089.08015,4594.02121,98977.97313,0.00000,988804,1.49367,0.06080,0.00465,0.00340,0.00312,0.38472,0.10010,0.00000
2,24257957.42603,3029090.47473,482292.55028,27155.55442,24983.10452,34335.32476,814936.80582,13.67732,7402335,3.27707,0.06515,0.00464,0.00367,0.00338,0.40921,0.11009,0.00000
3,3159139.42506,296745.04894,447471.08798,3219.74863,2962.16874,2478.92356,158883.37520,0.00000,542026,5.82839,0.82555,0.00457,0.00594,0.00546,0.54747,0.29313,0.00000
4,4482352.14201,535121.07565,826299.34603,1937.35233,1782.36415,1343.66918,90649.90240,25.10767,291556,15.37390,2.83410,0.00461,0.00664,0.00611,1.83540,0.31092,0.00009
5,2667721.62654,417654.10191,222165.58099,232.53766,213.93465,137.27460,8222.79065,837.78269,28848,92.47510,7.70125,0.00476,0.00806,0.00742,14.47775,0.28504,0.02904
X,174593.19662,29120.59356,7173.71365,246.36038,226.65155,320.64329,7429.40781,725.29966,68388,2.55298,0.10490,0.00469,0.00360,0.00331,0.42581,0.10864,0.01061


In [647]:
grp1_1 = adg.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp1_1

차종,승용,화물
등급,,
1,988804.00000,NaN
2,7382300.00000,20035.00000
3,541308.00000,718.00000
4,288876.00000,2680.00000
5,26955.00000,1893.00000
X,68120.00000,268.00000


In [648]:
grp2 = adl.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp2

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,459300.83780,7082.71807,47790.29873,1082.74333,996.12387,29041.00826,47326.08753,0.00000,476093
2,906574.90474,13610.94855,127323.88157,1937.17755,1782.20335,45766.86189,72150.04889,0.00000,740270
3,966636.95127,17605.51496,186389.06397,771.30234,709.59816,12111.01554,19742.34215,0.00000,201291
4,186604.18600,7156.06358,35026.61840,76.01085,69.92998,929.09047,1521.76821,0.00000,14390
5,10220.86967,895.49102,1574.18917,1.92488,1.77089,25.98582,42.34726,0.00000,428
X,7057.60040,106.48518,776.47572,15.63270,14.38209,421.40303,686.73086,0.00000,6690


In [649]:
grp2['CO'] = grp2['E_CO_total'] / grp2['차대번호']
grp2['NOx'] = grp2['E_NOx_total'] / grp2['차대번호']
grp2['SOx'] = grp2['E_SOx_total'] / grp2['차대번호']
grp2['PM10'] = grp2['E_PM10_total'] / grp2['차대번호']
grp2['PM2.5'] = grp2['E_PM2_5_total'] / grp2['차대번호']
grp2['HC'] = grp2['E_HC_total'] / grp2['차대번호']
grp2['NH3'] = grp2['E_NH3_total'] / grp2['차대번호']
grp2['CO2'] = grp2['E_CO2_total'] / grp2['차대번호']
grp2

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,459300.83780,7082.71807,47790.29873,1082.74333,996.12387,29041.00826,47326.08753,0.00000,476093,0.96473,0.10038,0.06100,0.00227,0.00209,0.01488,0.09941,0.00000
2,906574.90474,13610.94855,127323.88157,1937.17755,1782.20335,45766.86189,72150.04889,0.00000,740270,1.22465,0.17200,0.06182,0.00262,0.00241,0.01839,0.09746,0.00000
3,966636.95127,17605.51496,186389.06397,771.30234,709.59816,12111.01554,19742.34215,0.00000,201291,4.80219,0.92597,0.06017,0.00383,0.00353,0.08746,0.09808,0.00000
4,186604.18600,7156.06358,35026.61840,76.01085,69.92998,929.09047,1521.76821,0.00000,14390,12.96763,2.43409,0.06457,0.00528,0.00486,0.49729,0.10575,0.00000
5,10220.86967,895.49102,1574.18917,1.92488,1.77089,25.98582,42.34726,0.00000,428,23.88054,3.67801,0.06071,0.00450,0.00414,2.09227,0.09894,0.00000
X,7057.60040,106.48518,776.47572,15.63270,14.38209,421.40303,686.73086,0.00000,6690,1.05495,0.11607,0.06299,0.00234,0.00215,0.01592,0.10265,0.00000


In [650]:
grp2_1 = adl.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp2_1

차종,승용,화물
등급,,
1,476093.00000,NaN
2,695585.00000,44685.00000
3,200734.00000,557.00000
4,11670.00000,2720.00000
5,423.00000,5.00000
X,6348.00000,342.00000


In [651]:
grp3 = add.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp3

E_CO_total   E_HC_total    E_NOx_total  E_PM10_total  \
등급 저감장치부착유무                                                           
3  유          79007.50858  51399.39246 19380540.48335   18655.10768   
4  유           2594.43576   2772.15527  1692340.88177    4004.40392   
   NaN       111773.00948 112379.24538  3782633.98557  103504.75807   
5  유           2306.49698   9437.14982  1387985.20551    7895.94750   
   NaN      3555089.79259 812995.30296  8523864.34131  459243.75718   
X  유              2.18080      1.29319      408.34960       7.40956   
   NaN        21828.65361   1456.47484    32952.00535     742.66459   

             E_PM2_5_total  E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                                 
3  유           17118.53173 345575.16362  39019.05518 159032.04974  5213000  
4  유            3665.02586  19024.26735   1055.77460   9813.55376   226361  
   NaN         95149.14747  41731.92848   3421.11217   5148.24008   609544  
5  유            7562.76381  16922.45005    409.64224    607.00781    72050  
   NaN        431247.15141 103378.93255   3696.83532   2083.65150   673947  
X  유               7.74703      4.17925      0.36876      2.73755       82  
   NaN           702.73367   1434.19329    133.36737    274.06264    19393

In [652]:
grp3['CO'] = grp3['E_CO_total'] / grp3['차대번호']
grp3['NOx'] = grp3['E_NOx_total'] / grp3['차대번호']
grp3['SOx'] = grp3['E_SOx_total'] / grp3['차대번호']
grp3['PM10'] = grp3['E_PM10_total'] / grp3['차대번호']
grp3['PM2.5'] = grp3['E_PM2_5_total'] / grp3['차대번호']
grp3['HC'] = grp3['E_HC_total'] / grp3['차대번호']
grp3['NH3'] = grp3['E_NH3_total'] / grp3['차대번호']
grp3['CO2'] = grp3['E_CO2_total'] / grp3['차대번호']
grp3

E_CO_total   E_HC_total    E_NOx_total  E_PM10_total  \
등급 저감장치부착유무                                                           
3  유          79007.50858  51399.39246 19380540.48335   18655.10768   
4  유           2594.43576   2772.15527  1692340.88177    4004.40392   
   NaN       111773.00948 112379.24538  3782633.98557  103504.75807   
5  유           2306.49698   9437.14982  1387985.20551    7895.94750   
   NaN      3555089.79259 812995.30296  8523864.34131  459243.75718   
X  유              2.18080      1.29319      408.34960       7.40956   
   NaN        21828.65361   1456.47484    32952.00535     742.66459   

             E_PM2_5_total  E_SOx_total  E_NH3_total  E_CO2_total     차대번호  \
등급 저감장치부착유무                                                                  
3  유           17118.53173 345575.16362  39019.05518 159032.04974  5213000   
4  유            3665.02586  19024.26735   1055.77460   9813.55376   226361   
   NaN         95149.14747  41731.92848   3421.11217   5148.24008   609544   
5  유            7562.76381  16922.45005    409.64224    607.00781    72050   
   NaN        431247.15141 103378.93255   3696.83532   2083.65150   673947   
X  유               7.74703      4.17925      0.36876      2.73755       82   
   NaN           702.73367   1434.19329    133.36737    274.06264    19393   

                 CO      NOx     SOx    PM10   PM2.5      HC     NH3     CO2  
등급 저감장치부착유무                                                                   
3  유        0.01516  3.71773 0.06629 0.00358 0.00328 0.00986 0.00748 0.03051  
4  유        0.01146  7.47629 0.08404 0.01769 0.01619 0.01225 0.00466 0.04335  
   NaN      0.18337  6.20568 0.06846 0.16981 0.15610 0.18437 0.00561 0.00845  
5  유        0.03201 19.26419 0.23487 0.10959 0.10497 0.13098 0.00569 0.00842  
   NaN      5.27503 12.64768 0.15339 0.68142 0.63988 1.20632 0.00549 0.00309  
X  유        0.02660  4.97987 0.05097 0.09036 0.09448 0.01577 0.00450 0.03338  
   NaN      1.12559  1.69917 0.07395 0.03830 0.03624 0.07510 0.00688 0.01413

In [653]:
grp3_1 = add.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp3_1

차종                승용       화물
등급 저감장치부착유무                  
3  유         3222270  1990730
4  유          203984    22377
   NaN        148187   461357
5  유           21827    50223
   NaN        131619   542328
X  유              81        1
   NaN          7277    12116

In [654]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.03_PM.02.05.41'

In [655]:
with pd.ExcelWriter(os.path.join(ods_fold, f'통계_연료_등급별_배출가스배출량({today_date}).xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='휘발유')
    grp1_1.to_excel(writer, sheet_name='휘발유_차종')
    grp2.to_excel(writer, sheet_name='LPG')
    grp2_1.to_excel(writer, sheet_name='LPG_차종')
    grp3.to_excel(writer, sheet_name='경유')
    grp3_1.to_excel(writer, sheet_name='경유_차종')

## 배출량 통계2
- 조건
    - 승용(대형)
    - 화물(대형)

In [656]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '승용') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

3    1744356
5     175072
4     151014
X       8308
Name: 등급, dtype: int64

In [657]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '화물') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

3    189680
5     86825
4     21367
X      1080
Name: 등급, dtype: int64

In [658]:
ad2 = df2.loc[((df2['차종'] == '승용') & (df2['차종유형'] == '대형')) | ((df2['차종'] == '화물') & (df2['차종유형'] == '대형'))].reset_index(drop=True)
ad2.shape

(6662922, 70)

In [659]:
ad2g = ad2[ad2['fuel'] == '휘발유']
ad2l = ad2[ad2['fuel'] == 'LPG']
ad2d = ad2[ad2['fuel'] == '경유']
ad2.shape[0] == ad2g.shape[0] + ad2l.shape[0] + ad2d.shape[0]

True

In [660]:
grp21 = ad2g.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,1061184.11393,349641.79887,56026.26111,3121.45056,2871.73451,4246.36489,90885.88382,0.00000,911275
2,3183533.00642,1000678.64872,170697.01430,9480.39494,8721.96334,11985.00112,278857.70587,168.97383,2604532
3,1002362.21912,164479.39268,187986.04478,2033.97356,1871.25567,1615.28624,102710.23539,3.38713,355980
4,393530.59250,60593.21417,92470.20402,232.76806,214.14661,160.47756,10882.26093,0.00000,35399
5,81538.67186,13329.11334,7125.56065,7.30310,6.71885,5.06344,336.15767,0.00000,1106
X,57243.79687,15015.46032,4094.93753,111.82308,102.87724,146.92651,3455.85782,1.54467,31661


In [661]:
grp21['CO'] = grp21['E_CO_total'] / grp21['차대번호']
grp21['NOx'] = grp21['E_NOx_total'] / grp21['차대번호']
grp21['SOx'] = grp21['E_SOx_total'] / grp21['차대번호']
grp21['PM10'] = grp21['E_PM10_total'] / grp21['차대번호']
grp21['PM2.5'] = grp21['E_PM2_5_total'] / grp21['차대번호']
grp21['HC'] = grp21['E_HC_total'] / grp21['차대번호']
grp21['NH3'] = grp21['E_NH3_total'] / grp21['차대번호']
grp21['CO2'] = grp21['E_CO2_total'] / grp21['차대번호']
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,1061184.11393,349641.79887,56026.26111,3121.45056,2871.73451,4246.36489,90885.88382,0.00000,911275,1.16450,0.06148,0.00466,0.00343,0.00315,0.38368,0.09973,0.00000
2,3183533.00642,1000678.64872,170697.01430,9480.39494,8721.96334,11985.00112,278857.70587,168.97383,2604532,1.22231,0.06554,0.00460,0.00364,0.00335,0.38421,0.10707,0.00006
3,1002362.21912,164479.39268,187986.04478,2033.97356,1871.25567,1615.28624,102710.23539,3.38713,355980,2.81578,0.52808,0.00454,0.00571,0.00526,0.46205,0.28853,0.00001
4,393530.59250,60593.21417,92470.20402,232.76806,214.14661,160.47756,10882.26093,0.00000,35399,11.11700,2.61223,0.00453,0.00658,0.00605,1.71172,0.30742,0.00000
5,81538.67186,13329.11334,7125.56065,7.30310,6.71885,5.06344,336.15767,0.00000,1106,73.72393,6.44264,0.00458,0.00660,0.00607,12.05164,0.30394,0.00000
X,57243.79687,15015.46032,4094.93753,111.82308,102.87724,146.92651,3455.85782,1.54467,31661,1.80802,0.12934,0.00464,0.00353,0.00325,0.47426,0.10915,0.00005


In [662]:
grp21_1 = ad2g.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp21_1

차종,승용,화물
등급,,
1,911275.00000,NaN
2,2604522.00000,10.00000
3,355976.00000,4.00000
4,35395.00000,4.00000
5,1092.00000,14.00000
X,31637.00000,24.00000


In [663]:
grp22 = ad2l.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp22

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
2,339353.65194,5271.17526,36103.32023,779.70942,717.33266,19418.33187,31644.68897,0.00000,314737
3,131359.06244,2206.14641,24913.57614,112.91471,103.88153,1746.01852,2845.36351,0.00000,29005
4,10824.57001,384.47428,1787.92998,3.88419,3.57345,52.37255,85.32519,0.00000,871
5,45.77109,4.27800,7.78481,0.00950,0.00874,0.13878,0.20908,0.00000,2
X,630.58709,9.67205,66.01559,1.48201,1.36345,39.80713,64.87088,0.00000,652


In [664]:
grp22['CO'] = grp22['E_CO_total'] / grp22['차대번호']
grp22['NOx'] = grp22['E_NOx_total'] / grp22['차대번호']
grp22['SOx'] = grp22['E_SOx_total'] / grp22['차대번호']
grp22['PM10'] = grp22['E_PM10_total'] / grp22['차대번호']
grp22['PM2.5'] = grp22['E_PM2_5_total'] / grp22['차대번호']
grp22['HC'] = grp22['E_HC_total'] / grp22['차대번호']
grp22['NH3'] = grp22['E_NH3_total'] / grp22['차대번호']
grp22['CO2'] = grp22['E_CO2_total'] / grp22['차대번호']
grp22

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
2,339353.65194,5271.17526,36103.32023,779.70942,717.33266,19418.33187,31644.68897,0.00000,314737,1.07821,0.11471,0.06170,0.00248,0.00228,0.01675,0.10054,0.00000
3,131359.06244,2206.14641,24913.57614,112.91471,103.88153,1746.01852,2845.36351,0.00000,29005,4.52884,0.85894,0.06020,0.00389,0.00358,0.07606,0.09810,0.00000
4,10824.57001,384.47428,1787.92998,3.88419,3.57345,52.37255,85.32519,0.00000,871,12.42775,2.05273,0.06013,0.00446,0.00410,0.44142,0.09796,0.00000
5,45.77109,4.27800,7.78481,0.00950,0.00874,0.13878,0.20908,0.00000,2,22.88555,3.89241,0.06939,0.00475,0.00437,2.13900,0.10454,0.00000
X,630.58709,9.67205,66.01559,1.48201,1.36345,39.80713,64.87088,0.00000,652,0.96716,0.10125,0.06105,0.00227,0.00209,0.01483,0.09950,0.00000


In [665]:
grp22_1 = ad2l.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp22_1

차종,승용,화물
등급,,
2,314737.00000,NaN
3,29005.00000,NaN
4,870.00000,1.00000
5,1.00000,1.00000
X,652.00000,NaN


In [666]:
grp23 = ad2d.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp23

E_CO_total   E_HC_total   E_NOx_total  E_PM10_total  \
등급 저감장치부착유무                                                          
3  유          49679.97412  22017.89674 7237448.60584    6868.08598   
4  유            499.42302   1043.50525  518912.51237     736.11508   
   NaN        47804.74191  12218.86160  901518.48710    8401.02113   
5  유           3527.74063  14939.74278 2593111.42433   23545.53277   
   NaN      1181268.91328 327522.56010 6160248.32854  344159.92468   
X  유              2.22460      2.12071     524.63414       7.98647   
   NaN         8896.33470   1372.62500   38811.81040    1262.36898   

             E_PM2_5_total  E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                                 
3  유            6318.67661 190999.62005  24813.48654  26981.31542  1934036  
4  유             677.22587   9058.44132    851.14351    234.58775    24587  
   NaN          7729.31347  10012.16504    958.53864   2134.17044   147794  
5  유           22163.29788  16531.74718    634.28297    136.78118    57050  
   NaN        325686.00350  38337.89786   1695.74526    294.48650   204847  
X  유               8.20600      4.30576      0.33456      0.39241       69  
   NaN          1193.50295    997.43788    111.84451     37.37979     9319

In [667]:
grp23['CO'] = grp23['E_CO_total'] / grp23['차대번호']
grp23['NOx'] = grp23['E_NOx_total'] / grp23['차대번호']
grp23['SOx'] = grp23['E_SOx_total'] / grp23['차대번호']
grp23['PM10'] = grp23['E_PM10_total'] / grp23['차대번호']
grp23['PM2.5'] = grp23['E_PM2_5_total'] / grp23['차대번호']
grp23['HC'] = grp23['E_HC_total'] / grp23['차대번호']
grp23['NH3'] = grp23['E_NH3_total'] / grp23['차대번호']
grp23['CO2'] = grp23['E_CO2_total'] / grp23['차대번호']
grp23

E_CO_total   E_HC_total   E_NOx_total  E_PM10_total  \
등급 저감장치부착유무                                                          
3  유          49679.97412  22017.89674 7237448.60584    6868.08598   
4  유            499.42302   1043.50525  518912.51237     736.11508   
   NaN        47804.74191  12218.86160  901518.48710    8401.02113   
5  유           3527.74063  14939.74278 2593111.42433   23545.53277   
   NaN      1181268.91328 327522.56010 6160248.32854  344159.92468   
X  유              2.22460      2.12071     524.63414       7.98647   
   NaN         8896.33470   1372.62500   38811.81040    1262.36898   

             E_PM2_5_total  E_SOx_total  E_NH3_total  E_CO2_total     차대번호  \
등급 저감장치부착유무                                                                  
3  유            6318.67661 190999.62005  24813.48654  26981.31542  1934036   
4  유             677.22587   9058.44132    851.14351    234.58775    24587   
   NaN          7729.31347  10012.16504    958.53864   2134.17044   147794   
5  유           22163.29788  16531.74718    634.28297    136.78118    57050   
   NaN        325686.00350  38337.89786   1695.74526    294.48650   204847   
X  유               8.20600      4.30576      0.33456      0.39241       69   
   NaN          1193.50295    997.43788    111.84451     37.37979     9319   

                 CO      NOx     SOx    PM10   PM2.5      HC     NH3     CO2  
등급 저감장치부착유무                                                                   
3  유        0.02569  3.74215 0.09876 0.00355 0.00327 0.01138 0.01283 0.01395  
4  유        0.02031 21.10516 0.36842 0.02994 0.02754 0.04244 0.03462 0.00954  
   NaN      0.32346  6.09983 0.06774 0.05684 0.05230 0.08267 0.00649 0.01444  
5  유        0.06184 45.45331 0.28978 0.41272 0.38849 0.26187 0.01112 0.00240  
   NaN      5.76659 30.07244 0.18715 1.68008 1.58990 1.59886 0.00828 0.00144  
X  유        0.03224  7.60339 0.06240 0.11575 0.11893 0.03073 0.00485 0.00569  
   NaN      0.95464  4.16480 0.10703 0.13546 0.12807 0.14729 0.01200 0.00401

In [668]:
grp23_1 = ad2d.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp23_1

차종                승용      화물
등급 저감장치부착유무                 
3  유         1744356  189680
4  유            9144   15443
   NaN        141870    5924
5  유           30192   26858
   NaN        144880   59967
X  유              66       3
   NaN          8242    1077

In [669]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.03_PM.02.05.56'

In [670]:
with pd.ExcelWriter(os.path.join(ods_fold, f'통계_연료_등급별_배출가스배출량_대형({today_date}).xlsx')) as writer:
    grp21.to_excel(writer, sheet_name='휘발유')
    grp21_1.to_excel(writer, sheet_name='휘발유_차종')
    grp22.to_excel(writer, sheet_name='LPG')
    grp22_1.to_excel(writer, sheet_name='LPG_차종')
    grp23.to_excel(writer, sheet_name='경유')
    grp23_1.to_excel(writer, sheet_name='경유_차종')

## 배출량 통계3

In [149]:
# 1m 0.5s
csm = df2.loc[( (df2['차종'] == '승용') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) ) | 
             ( (df2['차종'] == '화물') & ((df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) )].reset_index(drop=True)
csm.shape

(17575496, 70)

In [150]:
# 9.3s
csmg = csm[csm['fuel'] == '휘발유']
csml = csm[csm['fuel'] == 'LPG']
csmd = csm[csm['fuel'] == '경유']
csm.shape[0] == csmg.shape[0] + csml.shape[0] + csmd.shape[0]

True

In [151]:
# 13.2s
large = df2.loc[((df2['차종'] == '승용') & (df2['차종유형'] == '대형')) | ((df2['차종'] == '화물') & (df2['차종유형'] == '대형'))].reset_index(drop=True)
large.shape

(6662922, 70)

In [152]:
# 3.3s
largeg = large[large['fuel'] == '휘발유']
largel = large[large['fuel'] == 'LPG']
larged = large[large['fuel'] == '경유']
large.shape[0] == largeg.shape[0] + largel.shape[0] + larged.shape[0]

True

### 경소중형

#### 휘발유

In [153]:
fuel = '휘발유'
div = 'HOT'
stat3_csmg_hot = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_hot = stat3_csmg_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_hot[f'E_{div}_CO_per'] = stat3_csmg_hot[f'E_{div}_CO'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_HC_per'] = stat3_csmg_hot[f'E_{div}_HC'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_NOx_per'] = stat3_csmg_hot[f'E_{div}_NOx'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_PM10_per'] = stat3_csmg_hot[f'E_{div}_PM10'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_PM2_5_per'] = stat3_csmg_hot[f'E_{div}_PM2_5'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,887832.70186,20059.45347,171731.57882,6084.55182,5597.78767,988804,13623.33933,0.89789,0.02029,0.17368,0.00615,0.00566
1,2,18040188.67322,461796.00008,1453743.45473,49210.88703,45273.99926,7402335,13579.61050,2.43709,0.06239,0.19639,0.00665,0.00612
2,3,6056760.89621,297349.90343,1289139.69146,5834.57478,5367.80880,542026,13400.23905,11.17430,0.54859,2.37837,0.01076,0.00990
3,4,10977854.22404,1100010.26227,2978978.87987,3511.07962,3230.19325,291556,13439.72625,37.65264,3.77290,10.21752,0.01204,0.01108
4,5,10028000.73690,1218841.98629,780345.94967,470.81512,433.14991,28848,13453.71806,347.61511,42.25048,27.05026,0.01632,0.01501
5,X,176158.02752,9245.59371,21183.11270,451.66583,415.53256,68388,13715.54001,2.57586,0.13519,0.30975,0.00660,0.00608


In [154]:
fuel = '휘발유'
div = 'COLD'
stat3_csmg_cold = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_cold = stat3_csmg_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_cold[f'E_{div}_CO_per'] = stat3_csmg_cold[f'E_{div}_CO'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_HC_per'] = stat3_csmg_cold[f'E_{div}_HC'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_NOx_per'] = stat3_csmg_cold[f'E_{div}_NOx'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_PM10_per'] = stat3_csmg_cold[f'E_{div}_PM10'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_PM2_5_per'] = stat3_csmg_cold[f'E_{div}_PM2_5'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,1684797.44679,59694.33522,121771.66993,0.00000,0.00000,988804,13623.33933,1.70387,0.06037,0.12315,0.00000,0.00000
1,2,30782042.25363,1156963.01974,902860.97429,0.00000,0.00000,7402335,13579.61050,4.15842,0.15630,0.12197,0.00000,0.00000
2,3,5103867.41929,330839.26250,428966.18396,0.00000,0.00000,542026,13400.23905,9.41628,0.61038,0.79141,0.00000,0.00000
3,4,8473657.12049,1130283.32560,844608.20168,0.00000,0.00000,291556,13439.72625,29.06357,3.87673,2.89690,0.00000,0.00000
4,5,2169386.41168,175383.59574,6850.74679,0.00000,0.00000,28848,13453.71806,75.20058,6.07958,0.23748,0.00000,0.00000
5,X,208178.66157,8525.71058,9565.26439,0.00000,0.00000,68388,13715.54001,3.04408,0.12467,0.13987,0.00000,0.00000


In [155]:
fuel = '휘발유'
div = 'EVA'
stat3_csmg_eva = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csmg_eva = stat3_csmg_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_eva[f'E_{div}_VOC_per'] = stat3_csmg_eva[f'E_{div}_VOC'] / stat3_csmg_eva['차량대수']
stat3_csmg_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,348432.71741,988804,13623.33933,0.35238
1,2,2606864.95841,7402335,13579.61050,0.35217
2,3,190426.38994,542026,13400.23905,0.35132
3,4,234925.00006,291556,13439.72625,0.80576
4,5,81895.84152,28848,13453.71806,2.83887
5,X,24401.63025,68388,13715.54001,0.35681


In [156]:
fuel = '휘발유'
div = 'total'
stat3_csmg_total = csmg.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_total = stat3_csmg_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_total[f'E_CO_{div}_per'] = stat3_csmg_total[f'E_CO_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_HC_{div}_per'] = stat3_csmg_total[f'E_HC_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_NOx_{div}_per'] = stat3_csmg_total[f'E_NOx_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_PM10_{div}_per'] = stat3_csmg_total[f'E_PM10_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_PM2_5_{div}_per'] = stat3_csmg_total[f'E_PM2_5_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_SOx_{div}_per'] = stat3_csmg_total[f'E_SOx_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_NH3_{div}_per'] = stat3_csmg_total[f'E_NH3_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_CO2_{div}_per'] = stat3_csmg_total[f'E_CO2_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,2572630.14865,428186.50611,293503.24875,6084.55182,5597.78767,13602.74556,153410.34806,90311.17856,988804,13623.33933,2.60176,0.43303,0.29683,0.00615,0.00566,0.01376,0.15515,0.09133
1,2,48822230.92685,4225623.97823,2356604.42902,49210.88703,45273.99926,101689.75421,1256687.67745,477583.02486,7402335,13579.61050,6.59552,0.57085,0.31836,0.00665,0.00612,0.01374,0.16977,0.06452
2,3,11160628.31550,818615.55586,1718105.87542,5834.57478,5367.80880,7340.84782,366656.75219,6072.57882,542026,13400.23905,20.59058,1.51029,3.16978,0.01076,0.00990,0.01354,0.67646,0.01120
3,4,19451511.34453,2465218.58793,3823587.08155,3511.07962,3230.19325,3981.81217,324759.06344,81.80036,291556,13439.72625,66.71621,8.45539,13.11442,0.01204,0.01108,0.01366,1.11388,0.00028
4,5,12197387.14858,1476121.42355,787196.69646,470.81512,433.14991,408.64122,28535.49495,314.74992,28848,13453.71806,422.81569,51.16893,27.28774,0.01632,0.01501,0.01417,0.98917,0.01091
5,X,384336.68909,42172.93455,30748.37709,451.66583,415.53256,949.75052,11958.10841,5620.29453,68388,13715.54001,5.61994,0.61667,0.44962,0.00660,0.00608,0.01389,0.17486,0.08218


#### LPG

In [157]:
fuel = 'LPG'
div = 'HOT'
stat3_csml_hot = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_hot = stat3_csml_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_hot[f'E_{div}_CO_per'] = stat3_csml_hot[f'E_{div}_CO'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_HC_per'] = stat3_csml_hot[f'E_{div}_HC'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_NOx_per'] = stat3_csml_hot[f'E_{div}_NOx'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_PM10_per'] = stat3_csml_hot[f'E_{div}_PM10'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_PM2_5_per'] = stat3_csml_hot[f'E_{div}_PM2_5'] / stat3_csml_hot['차량대수']
stat3_csml_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,481577.62887,13333.27350,138502.87997,1962.06216,1805.09719,476093,13692.16695,1.01152,0.02801,0.29092,0.00412,0.00379
1,2,1090703.27112,27613.37969,270302.09876,3510.40053,3229.56849,740270,13877.56522,1.47339,0.03730,0.36514,0.00474,0.00436
2,3,4003489.17967,120713.18252,938346.38244,1400.10788,1288.09925,201291,13505.43247,19.88906,0.59969,4.66164,0.00696,0.00640
3,4,689572.19444,37455.83171,163105.24313,164.43152,151.27699,14390,14492.70674,47.92024,2.60291,11.33462,0.01143,0.01051
4,5,41206.75853,5086.93955,7248.85613,3.48811,3.20906,428,13628.40070,96.27747,11.88537,16.93658,0.00815,0.00750
5,X,7947.47832,218.89766,2056.88717,28.32835,26.06208,6690,14139.16704,1.18796,0.03272,0.30746,0.00423,0.00390


In [158]:
fuel = 'LPG'
div = 'COLD'
stat3_csml_cold = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_cold = stat3_csml_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_cold[f'E_{div}_CO_per'] = stat3_csml_cold[f'E_{div}_CO'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_HC_per'] = stat3_csml_cold[f'E_{div}_HC'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_NOx_per'] = stat3_csml_cold[f'E_{div}_NOx'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_PM10_per'] = stat3_csml_cold[f'E_{div}_PM10'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_PM2_5_per'] = stat3_csml_cold[f'E_{div}_PM2_5'] / stat3_csml_cold['차량대수']
stat3_csml_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,204052.14008,1814.86141,0.00000,0.00000,0.00000,476093,13692.16695,0.42860,0.00381,0.00000,0.00000,0.00000
1,2,335515.72017,2917.74334,0.00000,0.00000,0.00000,740270,13877.56522,0.45323,0.00394,0.00000,0.00000,0.00000
2,3,938145.76108,8835.23449,0.00000,0.00000,0.00000,201291,13505.43247,4.66064,0.04389,0.00000,0.00000,0.00000
3,4,112491.59792,2011.71145,0.00000,0.00000,0.00000,14390,14492.70674,7.81735,0.13980,0.00000,0.00000,0.00000
4,5,8668.03395,343.74884,0.00000,0.00000,0.00000,428,13628.40070,20.25242,0.80315,0.00000,0.00000,0.00000
5,X,2789.07004,24.76368,0.00000,0.00000,0.00000,6690,14139.16704,0.41690,0.00370,0.00000,0.00000,0.00000


In [159]:
fuel = 'LPG'
div = 'EVA'
stat3_csml_eva = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csml_eva = stat3_csml_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_eva[f'E_{div}_VOC_per'] = stat3_csml_eva[f'E_{div}_VOC'] / stat3_csml_eva['차량대수']
stat3_csml_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,0.00000,476093,13692.16695,0.00000
1,2,0.00000,740270,13877.56522,0.00000
2,3,0.00000,201291,13505.43247,0.00000
3,4,0.00000,14390,14492.70674,0.00000
4,5,0.00000,428,13628.40070,0.00000
5,X,0.00000,6690,14139.16704,0.00000


In [160]:
fuel = 'LPG'
div = 'total'
stat3_csml_total = csml.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_total = stat3_csml_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_total[f'E_CO_{div}_per'] = stat3_csml_total[f'E_CO_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_HC_{div}_per'] = stat3_csml_total[f'E_HC_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_NOx_{div}_per'] = stat3_csml_total[f'E_NOx_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_PM10_{div}_per'] = stat3_csml_total[f'E_PM10_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_PM2_5_{div}_per'] = stat3_csml_total[f'E_PM2_5_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_SOx_{div}_per'] = stat3_csml_total[f'E_SOx_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_NH3_{div}_per'] = stat3_csml_total[f'E_NH3_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_CO2_{div}_per'] = stat3_csml_total[f'E_CO2_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,685629.76895,15148.13491,138502.87997,1962.06216,1805.09719,88003.05532,86021.35689,52383.34163,476093,13692.16695,1.44012,0.03182,0.29092,0.00412,0.00379,0.18484,0.18068,0.11003
1,2,1426218.99129,30531.12303,270302.09876,3510.40053,3229.56849,138687.46026,137438.45520,69458.65866,740270,13877.56522,1.92662,0.04124,0.36514,0.00474,0.00436,0.18735,0.18566,0.09383
2,3,4941634.94075,129548.41701,938346.38244,1400.10788,1288.09925,36700.04710,35909.95635,2128.78308,201291,13505.43247,24.54971,0.64359,4.66164,0.00696,0.00640,0.18232,0.17840,0.01058
3,4,802063.79236,39467.54316,163105.24313,164.43152,151.27699,2815.42567,2782.17183,0.00000,14390,14492.70674,55.73758,2.74271,11.33462,0.01143,0.01051,0.19565,0.19334,0.00000
4,5,49874.79248,5430.68839,7248.85613,3.48811,3.20906,78.74490,76.97168,0.00000,428,13628.40070,116.52989,12.68852,16.93658,0.00815,0.00750,0.18398,0.17984,0.00000
5,X,10736.54836,243.66134,2056.88717,28.32835,26.06208,1276.97887,1248.56736,695.97701,6690,14139.16704,1.60487,0.03642,0.30746,0.00423,0.00390,0.19088,0.18663,0.10403


#### 경유

In [161]:
fuel = '경유'
div = 'HOT'
stat3_csmd_hot = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_hot = stat3_csmd_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_hot[f'E_{div}_CO_per'] = stat3_csmd_hot[f'E_{div}_CO'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_HC_per'] = stat3_csmd_hot[f'E_{div}_HC'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_NOx_per'] = stat3_csmd_hot[f'E_{div}_NOx'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_PM10_per'] = stat3_csmd_hot[f'E_{div}_PM10'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_PM2_5_per'] = stat3_csmd_hot[f'E_{div}_PM2_5'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot

,등급,저감장치부착유무,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,3,유,39657.47350,100425.03879,52830833.48525,54656.18569,50191.65283,5213000,15321.84552,0.00761,0.01926,10.13444,0.01048,0.00963
1,4,유,1657.25907,8079.07443,6571254.73568,7702.14172,7047.91889,226361,14080.47617,0.00732,0.03569,29.02998,0.03403,0.03114
2,4,NaN,481683.99234,188556.62011,15788920.16196,414527.44060,381214.57233,609544,16621.54146,0.79024,0.30934,25.90284,0.68006,0.62541
3,5,유,7697.08090,40416.85876,4488332.96286,25904.17282,24075.59155,72050,17228.87033,0.10683,0.56096,62.29470,0.35953,0.33415
4,5,NaN,14571148.83929,3384493.91882,31160014.79769,1758208.67948,1629672.29491,673947,16622.27337,21.62062,5.02190,46.23511,2.60882,2.41810
5,X,유,0.75338,3.02360,1848.42228,10.47666,10.39848,82,13627.40854,0.00919,0.03687,22.54174,0.12776,0.12681
6,X,NaN,73358.66371,5819.15299,120796.80319,2629.11540,2444.69472,19393,16801.43389,3.78274,0.30006,6.22889,0.13557,0.12606


In [162]:
fuel = '경유'
div = 'COLD'
stat3_csmd_cold = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_cold = stat3_csmd_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_cold[f'E_{div}_CO_per'] = stat3_csmd_cold[f'E_{div}_CO'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_HC_per'] = stat3_csmd_cold[f'E_{div}_HC'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_NOx_per'] = stat3_csmd_cold[f'E_{div}_NOx'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_PM10_per'] = stat3_csmd_cold[f'E_{div}_PM10'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_PM2_5_per'] = stat3_csmd_cold[f'E_{div}_PM2_5'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold

,등급,저감장치부착유무,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,3,유,292482.14762,146279.45282,1243040.86010,16312.10820,15007.13954,5213000,15321.84552,0.05611,0.02806,0.23845,0.00313,0.00288
1,4,유,10954.21380,4585.99448,142126.04250,6518.20541,5996.74898,226361,14080.47617,0.04839,0.02026,0.62787,0.02880,0.02649
2,4,NaN,8292.00357,4166.65027,98967.78725,4283.14851,3940.57299,609544,16621.54146,0.01360,0.00684,0.16236,0.00703,0.00646
3,5,유,3147.70747,816.27708,14747.90731,2805.54393,2816.82875,72050,17228.87033,0.04369,0.01133,0.20469,0.03894,0.03910
4,5,NaN,23145.50241,5213.27044,92138.27338,20945.08879,21192.12910,673947,16622.27337,0.03434,0.00774,0.13671,0.03108,0.03144
5,X,유,10.91244,3.28405,56.94997,9.70557,9.66354,82,13627.40854,0.13308,0.04005,0.69451,0.11836,0.11785
6,X,NaN,804.62334,214.60400,2813.75228,104.51618,100.63114,19393,16801.43389,0.04149,0.01107,0.14509,0.00539,0.00519


In [163]:
fuel = '경유'
div = 'EVA'
stat3_csmd_eva = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csmd_eva = stat3_csmd_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_eva[f'E_{div}_VOC_per'] = stat3_csmd_eva[f'E_{div}_VOC'] / stat3_csmd_eva['차량대수']
stat3_csmd_eva

,등급,저감장치부착유무,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,3,유,0.00000,5213000,15321.84552,0.00000
1,4,유,0.00000,226361,14080.47617,0.00000
2,4,NaN,0.00000,609544,16621.54146,0.00000
3,5,유,0.00000,72050,17228.87033,0.00000
4,5,NaN,0.00000,673947,16622.27337,0.00000
5,X,유,0.00000,82,13627.40854,0.00000
6,X,NaN,0.00000,19393,16801.43389,0.00000


In [164]:
fuel = '경유'
div = 'total'
stat3_csmd_total = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_total = stat3_csmd_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_total[f'E_CO_{div}_per'] = stat3_csmd_total[f'E_CO_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_HC_{div}_per'] = stat3_csmd_total[f'E_HC_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_NOx_{div}_per'] = stat3_csmd_total[f'E_NOx_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_PM10_{div}_per'] = stat3_csmd_total[f'E_PM10_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_PM2_5_{div}_per'] = stat3_csmd_total[f'E_PM2_5_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_SOx_{div}_per'] = stat3_csmd_total[f'E_SOx_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_NH3_{div}_per'] = stat3_csmd_total[f'E_NH3_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_CO2_{div}_per'] = stat3_csmd_total[f'E_CO2_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total

,등급,저감장치부착유무,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,3,유,332139.62112,246704.49161,54073874.34535,70968.29390,65198.79237,1199270.76215,118239.56116,505910.51784,5213000,15321.84552,0.06371,0.04732,10.37289,0.01361,0.01251,0.23005,0.02268,0.09705
1,4,유,12611.47287,12665.06891,6713380.77818,14220.34713,13044.66786,61746.16836,3199.31696,31179.49394,226361,14080.47617,0.05571,0.05595,29.65785,0.06282,0.05763,0.27278,0.01413,0.13774
2,4,NaN,489975.99591,192723.27038,15887887.94921,418810.58910,385155.14532,146542.40173,10367.00657,14469.93794,609544,16621.54146,0.80384,0.31618,26.06520,0.68709,0.63187,0.24041,0.01701,0.02374
3,5,유,10844.78837,41233.13585,4503080.87017,28709.71675,26892.42029,49292.63199,1241.34011,2122.54725,72050,17228.87033,0.15052,0.57229,62.49939,0.39847,0.37325,0.68414,0.01723,0.02946
4,5,NaN,14594294.34171,3389707.18926,31252153.07106,1779153.76827,1650864.42401,313916.42627,11202.53127,5493.50109,673947,16622.27337,21.65496,5.02963,46.37183,2.63990,2.44955,0.46579,0.01662,0.00815
5,X,유,11.66583,6.30765,1905.37225,20.18223,20.06203,15.03882,1.11745,9.25469,82,13627.40854,0.14227,0.07692,23.23625,0.24612,0.24466,0.18340,0.01363,0.11286
6,X,NaN,74163.28705,6033.75699,123610.55547,2733.63158,2545.32585,4957.18751,404.14356,804.93862,19393,16801.43389,3.82423,0.31113,6.37398,0.14096,0.13125,0.25562,0.02084,0.04151


#### 출력(경소중형)

In [165]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.04_AM.03.15.59'

In [166]:
with pd.ExcelWriter(os.path.join(ods_fold, f'통계_연료_등급_엔진상태별_배출가스배출량_경소중형({today_date}).xlsx')) as writer:
    stat3_csmg_hot.to_excel(writer, sheet_name='휘발유_HOT', index=False)
    stat3_csmg_cold.to_excel(writer, sheet_name='휘발유_COLD', index=False)
    stat3_csmg_eva.to_excel(writer, sheet_name='휘발유_EVA', index=False)
    stat3_csmg_total.to_excel(writer, sheet_name='휘발유_TOTAL', index=False)
    stat3_csml_hot.to_excel(writer, sheet_name='LPG_HOT', index=False)
    stat3_csml_cold.to_excel(writer, sheet_name='LPG_COLD', index=False)
    stat3_csml_eva.to_excel(writer, sheet_name='LPG_EVA', index=False)
    stat3_csml_total.to_excel(writer, sheet_name='LPG_TOTAL', index=False)
    stat3_csmd_hot.to_excel(writer, sheet_name='경유_HOT', index=False)
    stat3_csmd_cold.to_excel(writer, sheet_name='경유_COLD', index=False)
    stat3_csmd_eva.to_excel(writer, sheet_name='경유_EVA', index=False)
    stat3_csmd_total.to_excel(writer, sheet_name='경유_TOTAL', index=False)

### 대형

#### 휘발유

In [167]:
fuel = '휘발유'
div = 'HOT'
stat3_largeg_hot = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_hot = stat3_largeg_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_hot[f'E_{div}_CO_per'] = stat3_largeg_hot[f'E_{div}_CO'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_HC_per'] = stat3_largeg_hot[f'E_{div}_HC'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_NOx_per'] = stat3_largeg_hot[f'E_{div}_NOx'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_PM10_per'] = stat3_largeg_hot[f'E_{div}_PM10'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_PM2_5_per'] = stat3_largeg_hot[f'E_{div}_PM2_5'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,1007093.07379,16749.82396,160182.86505,5656.44677,5203.93103,911275,13663.71034,1.10515,0.01838,0.17578,0.00621,0.00571
1,2,3182204.25813,53431.88702,503669.31592,17179.62477,15805.25478,2604532,13492.98804,1.22180,0.02051,0.19338,0.00660,0.00607
2,3,2120488.46960,96716.69494,559389.57174,3685.80663,3390.94210,355980,13305.21074,5.95676,0.27169,1.57141,0.01035,0.00953
3,4,1229164.17238,124349.38468,349082.13472,421.80393,388.05962,35399,13292.08660,34.72313,3.51279,9.86136,0.01192,0.01096
4,5,306792.53617,37561.16306,25661.41996,13.23410,12.17537,1106,13339.72694,277.38927,33.96127,23.20201,0.01197,0.01101
5,X,110578.80598,9316.39477,12829.93444,202.63698,186.42602,31661,13601.52596,3.49259,0.29425,0.40523,0.00640,0.00589


In [168]:
fuel = '휘발유'
div = 'COLD'
stat3_largeg_cold = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_cold = stat3_largeg_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_cold[f'E_{div}_CO_per'] = stat3_largeg_cold[f'E_{div}_CO'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_HC_per'] = stat3_largeg_cold[f'E_{div}_HC'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_NOx_per'] = stat3_largeg_cold[f'E_{div}_NOx'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_PM10_per'] = stat3_largeg_cold[f'E_{div}_PM10'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_PM2_5_per'] = stat3_largeg_cold[f'E_{div}_PM2_5'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,0.00000,0.00000,0.00000,0.00000,0.00000,911275,13663.71034,0.00000,0.00000,0.00000,0.00000,0.00000
1,2,0.00000,0.00000,0.00000,0.00000,0.00000,2604532,13492.98804,0.00000,0.00000,0.00000,0.00000,0.00000
2,3,0.00000,0.00000,0.00000,0.00000,0.00000,355980,13305.21074,0.00000,0.00000,0.00000,0.00000,0.00000
3,4,0.00000,0.00000,0.00000,0.00000,0.00000,35399,13292.08660,0.00000,0.00000,0.00000,0.00000,0.00000
4,5,0.00000,0.00000,0.00000,0.00000,0.00000,1106,13339.72694,0.00000,0.00000,0.00000,0.00000,0.00000
5,X,0.00000,0.00000,0.00000,0.00000,0.00000,31661,13601.52596,0.00000,0.00000,0.00000,0.00000,0.00000


In [169]:
fuel = '휘발유'
div = 'EVA'
stat3_largeg_eva = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_largeg_eva = stat3_largeg_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_eva[f'E_{div}_VOC_per'] = stat3_largeg_eva[f'E_{div}_VOC'] / stat3_largeg_eva['차량대수']
stat3_largeg_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,321290.18254,911275,13663.71034,0.35257
1,2,916158.55271,2604532,13492.98804,0.35176
2,3,124899.67508,355980,13305.21074,0.35086
3,4,29886.27473,35399,13292.08660,0.84427
4,5,3119.52973,1106,13339.72694,2.82055
5,X,11609.34232,31661,13601.52596,0.36668


In [170]:
fuel = '휘발유'
div = 'total'
stat3_largeg_total = largeg.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_total = stat3_largeg_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_total[f'E_CO_{div}_per'] = stat3_largeg_total[f'E_CO_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_HC_{div}_per'] = stat3_largeg_total[f'E_HC_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_NOx_{div}_per'] = stat3_largeg_total[f'E_NOx_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_PM10_{div}_per'] = stat3_largeg_total[f'E_PM10_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_PM2_5_{div}_per'] = stat3_largeg_total[f'E_PM2_5_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_SOx_{div}_per'] = stat3_largeg_total[f'E_SOx_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_NH3_{div}_per'] = stat3_largeg_total[f'E_NH3_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_CO2_{div}_per'] = stat3_largeg_total[f'E_CO2_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,1007093.07379,338040.00649,160182.86505,5656.44677,5203.93103,12573.34663,140918.10505,0.00000,911275,13663.71034,1.10515,0.37095,0.17578,0.00621,0.00571,0.01380,0.15464,0.00000
1,2,3182204.25813,969590.43973,503669.31592,17179.62477,15805.25478,35487.20644,430460.50381,835.80965,2604532,13492.98804,1.22180,0.37227,0.19338,0.00660,0.00607,0.01363,0.16527,0.00032
2,3,2120488.46960,221616.37003,559389.57174,3685.80663,3390.94210,4782.81420,203379.10412,21.91073,355980,13305.21074,5.95676,0.62255,1.57141,0.01035,0.00953,0.01344,0.57132,0.00006
3,4,1229164.17238,154235.65942,349082.13472,421.80393,388.05962,475.17383,37865.28998,0.00000,35399,13292.08660,34.72313,4.35706,9.86136,0.01192,0.01096,0.01342,1.06967,0.00000
4,5,306792.53617,40680.69279,25661.41996,13.23410,12.17537,15.00680,1168.65317,0.00000,1106,13339.72694,277.38927,36.78182,23.20201,0.01197,0.01101,0.01357,1.05665,0.00000
5,X,110578.80598,20925.73709,12829.93444,202.63698,186.42602,435.07280,5624.72430,6.77609,31661,13601.52596,3.49259,0.66093,0.40523,0.00640,0.00589,0.01374,0.17765,0.00021


#### LPG

In [171]:
fuel = 'LPG'
div = 'HOT'
stat3_largel_hot = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_hot = stat3_largel_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_hot[f'E_{div}_CO_per'] = stat3_largel_hot[f'E_{div}_CO'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_HC_per'] = stat3_largel_hot[f'E_{div}_HC'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_NOx_per'] = stat3_largel_hot[f'E_{div}_NOx'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_PM10_per'] = stat3_largel_hot[f'E_{div}_PM10'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_PM2_5_per'] = stat3_largel_hot[f'E_{div}_PM2_5'] / stat3_largel_hot['차량대수']
stat3_largel_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,2,398874.81572,10616.61617,107086.45117,1412.92797,1299.89373,314737,13848.93605,1.26733,0.03373,0.34024,0.00449,0.00413
1,3,583956.24825,15988.28901,126134.43065,204.61514,188.24593,29005,13512.26854,20.13295,0.55123,4.34871,0.00705,0.00649
2,4,41047.10453,2105.08396,9294.47453,7.05970,6.49493,871,13493.43398,47.12641,2.41686,10.67104,0.00811,0.00746
3,5,203.81263,25.16794,35.84553,0.01722,0.01584,2,14399.25000,101.90632,12.58397,17.92276,0.00861,0.00792
4,X,707.10196,19.00381,194.58067,2.68558,2.47073,652,13704.57439,1.08451,0.02915,0.29844,0.00412,0.00379


In [172]:
fuel = 'LPG'
div = 'COLD'
stat3_largel_cold = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_cold = stat3_largel_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_cold[f'E_{div}_CO_per'] = stat3_largel_cold[f'E_{div}_CO'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_HC_per'] = stat3_largel_cold[f'E_{div}_HC'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_NOx_per'] = stat3_largel_cold[f'E_{div}_NOx'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_PM10_per'] = stat3_largel_cold[f'E_{div}_PM10'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_PM2_5_per'] = stat3_largel_cold[f'E_{div}_PM2_5'] / stat3_largel_cold['차량대수']
stat3_largel_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,2,0.00000,0.00000,0.00000,0.00000,0.00000,314737,13848.93605,0.00000,0.00000,0.00000,0.00000,0.00000
1,3,0.00000,0.00000,0.00000,0.00000,0.00000,29005,13512.26854,0.00000,0.00000,0.00000,0.00000,0.00000
2,4,0.00000,0.00000,0.00000,0.00000,0.00000,871,13493.43398,0.00000,0.00000,0.00000,0.00000,0.00000
3,5,0.00000,0.00000,0.00000,0.00000,0.00000,2,14399.25000,0.00000,0.00000,0.00000,0.00000,0.00000
4,X,0.00000,0.00000,0.00000,0.00000,0.00000,652,13704.57439,0.00000,0.00000,0.00000,0.00000,0.00000


In [173]:
fuel = 'LPG'
div = 'EVA'
stat3_largel_eva = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_largel_eva = stat3_largel_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_eva[f'E_{div}_VOC_per'] = stat3_largel_eva[f'E_{div}_VOC'] / stat3_largel_eva['차량대수']
stat3_largel_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,2,0.00000,314737,13848.93605,0.00000
1,3,0.00000,29005,13512.26854,0.00000
2,4,0.00000,871,13493.43398,0.00000
3,5,0.00000,2,14399.25000,0.00000
4,X,0.00000,652,13704.57439,0.00000


In [174]:
fuel = 'LPG'
div = 'total'
stat3_largel_total = largel.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_total = stat3_largel_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_total[f'E_CO_{div}_per'] = stat3_largel_total[f'E_CO_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_HC_{div}_per'] = stat3_largel_total[f'E_HC_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_NOx_{div}_per'] = stat3_largel_total[f'E_NOx_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_PM10_{div}_per'] = stat3_largel_total[f'E_PM10_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_PM2_5_{div}_per'] = stat3_largel_total[f'E_PM2_5_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_SOx_{div}_per'] = stat3_largel_total[f'E_SOx_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_NH3_{div}_per'] = stat3_largel_total[f'E_NH3_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_CO2_{div}_per'] = stat3_largel_total[f'E_CO2_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,2,398874.81572,10616.61617,107086.45117,1412.92797,1299.89373,58843.42990,57518.36304,37128.44496,314737,13848.93605,1.26733,0.03373,0.34024,0.00449,0.00413,0.18696,0.18275,0.11797
1,3,583956.24825,15988.28901,126134.43065,204.61514,188.24593,5290.96521,5171.82051,0.19366,29005,13512.26854,20.13295,0.55123,4.34871,0.00705,0.00649,0.18242,0.17831,0.00001
2,4,41047.10453,2105.08396,9294.47453,7.05970,6.49493,158.70470,155.08970,0.00000,871,13493.43398,47.12641,2.41686,10.67104,0.00811,0.00746,0.18221,0.17806,0.00000
3,5,203.81263,25.16794,35.84553,0.01722,0.01584,0.42056,0.38003,0.00000,2,14399.25000,101.90632,12.58397,17.92276,0.00861,0.00792,0.21028,0.19001,0.00000
4,X,707.10196,19.00381,194.58067,2.68558,2.47073,120.62766,117.91131,50.15584,652,13704.57439,1.08451,0.02915,0.29844,0.00412,0.00379,0.18501,0.18085,0.07693


#### 경유

In [175]:
fuel = '경유'
div = 'HOT'
stat3_larged_hot = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_hot = stat3_larged_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_hot[f'E_{div}_CO_per'] = stat3_larged_hot[f'E_{div}_CO'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_HC_per'] = stat3_larged_hot[f'E_{div}_HC'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_NOx_per'] = stat3_larged_hot[f'E_{div}_NOx'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_PM10_per'] = stat3_larged_hot[f'E_{div}_PM10'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_PM2_5_per'] = stat3_larged_hot[f'E_{div}_PM2_5'] / stat3_larged_hot['차량대수']
stat3_larged_hot

,등급,저감장치부착유무,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,3,유,26793.62419,42161.43429,24122847.36464,17922.13908,16488.39860,1934036,14224.88439,0.01385,0.02180,12.47280,0.00927,0.00853
1,4,유,1480.78808,4461.65235,2114545.98429,2643.12842,2431.67815,24587,16815.26957,0.06023,0.18146,86.00260,0.10750,0.09890
2,4,NaN,206754.80966,50704.35289,3636230.95948,28055.09266,25811.20391,147794,13696.50979,1.39894,0.34307,24.60337,0.18983,0.17464
3,5,유,10901.37947,63268.22418,10233512.44142,83514.10877,77242.60885,57050,15838.80872,0.19108,1.10900,179.37796,1.46388,1.35395
4,5,NaN,4969757.57954,1406477.25987,24479298.99157,1247791.51479,1160529.99598,204847,14776.13553,24.26083,6.86599,119.50040,6.09133,5.66535
5,X,유,1.53445,7.09752,2258.34084,15.41998,14.88770,69,13587.52174,0.02224,0.10286,32.72958,0.22348,0.21576
6,X,NaN,34849.61081,5828.54346,148529.34896,4503.11582,4187.41172,9319,14533.63494,3.73963,0.62545,15.93834,0.48322,0.44934


In [176]:
fuel = '경유'
div = 'COLD'
stat3_larged_cold = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_cold = stat3_larged_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_cold[f'E_{div}_CO_per'] = stat3_larged_cold[f'E_{div}_CO'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_HC_per'] = stat3_larged_cold[f'E_{div}_HC'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_NOx_per'] = stat3_larged_cold[f'E_{div}_NOx'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_PM10_per'] = stat3_larged_cold[f'E_{div}_PM10'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_PM2_5_per'] = stat3_larged_cold[f'E_{div}_PM2_5'] / stat3_larged_cold['차량대수']
stat3_larged_cold

,등급,저감장치부착유무,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,3,유,0.00000,0.00000,0.00000,0.00000,0.00000,1934036,14224.88439,0.00000,0.00000,0.00000,0.00000,0.00000
1,4,유,0.00000,0.00000,0.00000,0.00000,0.00000,24587,16815.26957,0.00000,0.00000,0.00000,0.00000,0.00000
2,4,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,147794,13696.50979,0.00000,0.00000,0.00000,0.00000,0.00000
3,5,유,0.00000,0.00000,0.00000,0.00000,0.00000,57050,15838.80872,0.00000,0.00000,0.00000,0.00000,0.00000
4,5,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,204847,14776.13553,0.00000,0.00000,0.00000,0.00000,0.00000
5,X,유,0.00000,0.00000,0.00000,0.00000,0.00000,69,13587.52174,0.00000,0.00000,0.00000,0.00000,0.00000
6,X,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,9319,14533.63494,0.00000,0.00000,0.00000,0.00000,0.00000


In [177]:
fuel = '경유'
div = 'EVA'
stat3_larged_eva = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_larged_eva = stat3_larged_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_eva[f'E_{div}_VOC_per'] = stat3_larged_eva[f'E_{div}_VOC'] / stat3_larged_eva['차량대수']
stat3_larged_eva

,등급,저감장치부착유무,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,3,유,0.00000,1934036,14224.88439,0.00000
1,4,유,0.00000,24587,16815.26957,0.00000
2,4,NaN,0.00000,147794,13696.50979,0.00000
3,5,유,0.00000,57050,15838.80872,0.00000
4,5,NaN,0.00000,204847,14776.13553,0.00000
5,X,유,0.00000,69,13587.52174,0.00000
6,X,NaN,0.00000,9319,14533.63494,0.00000


In [178]:
fuel = '경유'
div = 'total'
stat3_larged_total = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_total = stat3_larged_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_total[f'E_CO_{div}_per'] = stat3_larged_total[f'E_CO_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_HC_{div}_per'] = stat3_larged_total[f'E_HC_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_NOx_{div}_per'] = stat3_larged_total[f'E_NOx_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_PM10_{div}_per'] = stat3_larged_total[f'E_PM10_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_PM2_5_{div}_per'] = stat3_larged_total[f'E_PM2_5_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_SOx_{div}_per'] = stat3_larged_total[f'E_SOx_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_NH3_{div}_per'] = stat3_larged_total[f'E_NH3_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_CO2_{div}_per'] = stat3_larged_total[f'E_CO2_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total

,등급,저감장치부착유무,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,3,유,26793.62419,42161.43429,24122847.36464,17922.13908,16488.39860,637337.58104,75192.38346,318767.78507,1934036,14224.88439,0.01385,0.02180,12.47280,0.00927,0.00853,0.32954,0.03888,0.16482
1,4,유,1480.78808,4461.65235,2114545.98429,2643.12842,2431.67815,28190.09655,2579.22274,8348.41671,24587,16815.26957,0.06023,0.18146,86.00260,0.10750,0.09890,1.14654,0.10490,0.33955
2,4,NaN,206754.80966,50704.35289,3636230.95948,28055.09266,25811.20391,34739.18491,2904.66255,5696.44557,147794,13696.50979,1.39894,0.34307,24.60337,0.18983,0.17464,0.23505,0.01965,0.03854
3,5,유,10901.37947,63268.22418,10233512.44142,83514.10877,77242.60885,51787.83478,1922.06960,5238.54214,57050,15838.80872,0.19108,1.10900,179.37796,1.46388,1.35395,0.90776,0.03369,0.09182
4,5,NaN,4969757.57954,1406477.25987,24479298.99157,1247791.51479,1160529.99598,122226.43191,5138.62199,5281.88540,204847,14776.13553,24.26083,6.86599,119.50040,6.09133,5.66535,0.59667,0.02509,0.02578
5,X,유,1.53445,7.09752,2258.34084,15.41998,14.88770,15.09278,1.01382,7.87446,69,13587.52174,0.02224,0.10286,32.72958,0.22348,0.21576,0.21874,0.01469,0.11412
6,X,NaN,34849.61081,5828.54346,148529.34896,4503.11582,4187.41172,3309.49584,338.92276,966.96447,9319,14533.63494,3.73963,0.62545,15.93834,0.48322,0.44934,0.35513,0.03637,0.10376


#### 출력(대형)

In [179]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.04_AM.03.16.02'

In [180]:
with pd.ExcelWriter(os.path.join(ods_fold, f'통계_연료_등급_엔진상태별_배출가스배출량_대형({today_date}).xlsx')) as writer:
    stat3_largeg_hot.to_excel(writer, sheet_name='휘발유_HOT', index=False)
    stat3_largeg_cold.to_excel(writer, sheet_name='휘발유_COLD', index=False)
    stat3_largeg_eva.to_excel(writer, sheet_name='휘발유_EVA', index=False)
    stat3_largeg_total.to_excel(writer, sheet_name='휘발유_TOTAL', index=False)
    stat3_largel_hot.to_excel(writer, sheet_name='LPG_HOT', index=False)
    stat3_largel_cold.to_excel(writer, sheet_name='LPG_COLD', index=False)
    stat3_largel_eva.to_excel(writer, sheet_name='LPG_EVA', index=False)
    stat3_largel_total.to_excel(writer, sheet_name='LPG_TOTAL', index=False)
    stat3_larged_hot.to_excel(writer, sheet_name='경유_HOT', index=False)
    stat3_larged_cold.to_excel(writer, sheet_name='경유_COLD', index=False)
    stat3_larged_eva.to_excel(writer, sheet_name='경유_EVA', index=False)
    stat3_larged_total.to_excel(writer, sheet_name='경유_TOTAL', index=False)

## 총 배출량 통계

In [181]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [182]:
grp3 = df2.groupby('fuel').agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum'})
grp3

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
fuel,,,,,,,,
LPG,13405379.43162,457443.55048,2376106.57349,14823.68012,13691.03830,365081.27039,358884.95585,161850.75152
경유,28043282.44125,7391695.18458,218496020.19398,4447757.15475,4186992.32797,3502315.27082,280938.98268,942530.15998
휘발유,103107095.07904,11284208.34461,10765698.62677,92878.67924,85448.36215,181984.62690,2968821.04327,580866.76121


In [183]:
grp3_sum = grp3.sum().reset_index().T
grp3_sum.columns = grp3_sum.iloc[0, :]
grp3_sum.index = ['index', '합계']
grp3_sum = grp3_sum.iloc[[1]]
grp3_sum

index,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
합계,144555756.95191,19133347.07968,231637825.39424,4555459.51412,4286131.72842,4049381.16811,3608644.98181,1685247.67271


In [184]:
grp3m = pd.concat([grp3, grp3_sum])
grp3m

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
LPG,13405379.43162,457443.55048,2376106.57349,14823.68012,13691.03830,365081.27039,358884.95585,161850.75152
경유,28043282.44125,7391695.18458,218496020.19398,4447757.15475,4186992.32797,3502315.27082,280938.98268,942530.15998
휘발유,103107095.07904,11284208.34461,10765698.62677,92878.67924,85448.36215,181984.62690,2968821.04327,580866.76121
합계,144555756.95191,19133347.07968,231637825.39424,4555459.51412,4286131.72842,4049381.16811,3608644.98181,1685247.67271


In [185]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.04_AM.03.16.04'

In [186]:
grp3m.to_excel(os.path.join(ods_fold, f'통계_CAPSS_배출량({today_date}).xlsx'))

## 등급, 연료별 차량대수

In [713]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [714]:
df2['fuel'].value_counts(dropna=False)

휘발유    13272033
경유      9926539
LPG     1918526
Name: fuel, dtype: int64

In [715]:
car_stat = df2.groupby(['등급', 'fuel'], dropna=False)['차대번호'].count().unstack('fuel')
car_stat

fuel,LPG,경유,휘발유
등급,,,
1,476093.00,NaN,1900079.00
2,1171739.00,NaN,10011174.00
3,236585.00,7618188.00,898306.00
4,26089.00,1152632.00,332311.00
5,436.00,1124165.00,30040.00
X,7584.00,31554.00,100123.00


In [716]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.07.03_PM.03.50.30'

In [717]:
car_stat.to_excel(os.path.join(ods_fold, f'통계_등급_연료별_차량대수({today_date}).xlsx'))

# 코드 마지막